In [ ]:
!pip install  -U -q trl peft math_verify
# Tested with transformers==4.47.1, trl==0.14.0, datasets==3.2.0, peft==0.14.0, accelerate==1.2.1, math_verify==0.3.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login


In [ ]:
from datasets import load_dataset

dataset_id = "AI-MO/NuminaMath-TIR"
train_dataset, test_dataset = load_dataset(dataset_id, split=["train[:5%]", "test[:5%]"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/215k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/72441 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset)

Dataset({
    features: ['problem', 'solution', 'messages'],
    num_rows: 3622
})


In [ ]:
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)


def make_conversation(example):
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["problem"]},
        ],
    }


train_dataset = train_dataset.map(make_conversation)
test_dataset = test_dataset.map(make_conversation)

Map:   0%|          | 0/3622 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.remove_columns(["messages", "problem"])
print(train_dataset)

Dataset({
    features: ['solution', 'prompt'],
    num_rows: 3622
})


In [ ]:
import torch
from transformers import AutoModelForCausalLM

model_id = "Qwen/Qwen2-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


In [ ]:
import re


def format_reward(completions, **kwargs):
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<think>.*?</think>\s*<answer>.*?</answer>$"
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content) for content in completion_contents]
    rewards_list = [1.0 if match else 0.0 for match in matches]
    return [1.0 if match else 0.0 for match in matches]

In [ ]:
from math_verify import LatexExtractionConfig, parse, verify


def accuracy_reward(completions, **kwargs):
    """Reward function that checks if the completion is the same as the ground truth."""
    solutions = kwargs["solution"]
    completion_contents = [completion[0]["content"] for completion in completions]
    rewards = []
    for content, solution in zip(completion_contents, solutions):
        gold_parsed = parse(solution, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
        answer_parsed = parse(content, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
        if len(gold_parsed) != 0:
            try:
                rewards.append(float(verify(answer_parsed, gold_parsed)))
            except Exception:
                rewards.append(0.0)
        else:
            rewards.append(1.0)
    return rewards

In [ ]:
from trl import GRPOConfig

# Configure training arguments using GRPOConfig
training_args = GRPOConfig(
    output_dir="Qwen2-0.5B-GRPO-test",
    learning_rate=1e-5,
    remove_unused_columns=False,  # to access the solution column in accuracy_reward
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    bf16=True,
    # Parameters that control de data preprocessing
    max_completion_length=64,  # default: 256
    num_generations=4,  # default: 8
    max_prompt_length=128,  # default: 512
    # Parameters related to reporting and saving
    report_to=["tensorboard"],
    logging_steps=10,
    push_to_hub=False,
    save_strategy="steps",
    save_steps=10,
)

In [ ]:
from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model, reward_funcs=[format_reward, accuracy_reward], args=training_args, train_dataset=train_dataset
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss


ERROR:math_verify.grader:Error during comparison
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/math_verify/grader.py", line 809, in compare_single_extraction_wrapper
    return compare_single_extraction(g, t)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/math_verify/utils.py", line 51, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/math_verify/grader.py", line 789, in compare_single_extraction
    return sympy_expr_eq(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/math_verify/grader.py", line 667, in sympy_expr_eq
    return sympy_compare_relational(gold, pred, float_rounding, numeric_precision)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/math_verify/grader.py", line 344, in sympy_compare_relational
    if sympy_sol

KeyboardInterrupt: 

#add Data Parallel and Model parallel pipeline? - cant use anything can just increase batch size



In [ ]:
from trl import GRPOConfig

training_args = GRPOConfig(
    # 1) Mixed precision
    bf16=True,              # or fp16=True on non-bf16 cards

    # 2) Bigger per‐GPU batch, fewer accumulations
    per_device_train_batch_size=16,
    gradient_accumulation_steps=16,

    # 3) Leaner reward eval
    num_generations=2,
    max_prompt_length=128,
    max_completion_length=64,

    # 4) Lighten I/O
    logging_steps=100,
    save_strategy="epoch",

    # keep your other args…
    learning_rate=1e-5,
    num_train_epochs=1,
    remove_unused_columns=False,
    report_to=["tensorboard"],
    push_to_hub=False,
)


In [ ]:
from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model, reward_funcs=[format_reward, accuracy_reward], args=training_args, train_dataset=train_dataset
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss


ERROR:math_verify.grader:Error during comparison
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/math_verify/grader.py", line 809, in compare_single_extraction_wrapper
    return compare_single_extraction(g, t)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/math_verify/utils.py", line 51, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/math_verify/grader.py", line 789, in compare_single_extraction
    return sympy_expr_eq(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/math_verify/grader.py", line 667, in sympy_expr_eq
    return sympy_compare_relational(gold, pred, float_rounding, numeric_precision)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/math_verify/grader.py", line 344, in sympy_compare_relational
    if sympy_sol

TrainOutput(global_step=28, training_loss=0.007950196308749062, metrics={'train_runtime': 3854.5149, 'train_samples_per_second': 0.94, 'train_steps_per_second': 0.007, 'total_flos': 0.0, 'train_loss': 0.007950196308749062})

In [ ]:
trainer.save_model(model)


TypeError: expected str, bytes or os.PathLike object, not PeftModelForCausalLM

In [ ]:
# 1. Zip up the model directory
import shutil
shutil.make_archive("my_model", "zip", training_args.output_dir)

# 2. Download the zip file to your local machine
from google.colab import files
files.download("my_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Reloading the model from drive

In [ ]:
# 1. Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# 2. Set the path to your model folder
model_path = '/content/drive/MyDrive/GPRO_3b/my_model'

# 3. Load tokenizer and model (adjust load_in_8bit/device_map if you quantized it)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map='auto',       # or {"": 0} to force GPU 0
    load_in_8bit=False        # if you used bitsandbytes quantization
)

# 4. Create a text-generation pipeline
gen = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    # device=0                  # GPU device index
)

# 5. Run a prompt through your model
prompt = "Explain Group Relative Policy Optimization in simple terms:"
outputs = gen(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)

# 6. Print the generated text
print(outputs[0]['generated_text'])


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
Device set to use cuda:0


Explain Group Relative Policy Optimization in simple terms: Group relative policy optimization refers to a technique used in the field of computer science and software engineering where a group of individuals or entities work together to develop, maintain, and improve a piece of software or hardware. It involves assigning tasks to different members of the group based on their expertise and availability, and then using machine learning algorithms to optimize the performance of the software or hardware.
For example, if you were working on developing a new mobile app for your company, you might assign one person to design the user


In [ ]:
prompt ="In 1988, a person's age was equal to the sum of the digits of their birth year. How old was this person?"
outputs = gen(prompt, max_new_tokens=1000, do_sample=True, temperature=0.7)

# 6. Print the generated text
print(outputs[0]['generated_text'])

In 1988, a person's age was equal to the sum of the digits of their birth year. How old was this person?

I know that $10^{2n} - 1$ is divisible by $4$, but I don't know how to prove it.



This is just a special case of Fermat's little theorem. It states that if $a$ and $b$ are coprime integers then $a^p \equiv a \mod p$ for all primes $p$. 

So in your problem, we have that $10^n-1$ is divisible by 4 because 10 and 4 are relatively prime (since they are both odd). But $10^n=2\cdot5^n$ so $5|2\cdot5^n$ which means that $5|10^n-1$ because $5$ divides any power of 10 except for $5^n$. So $10^n-1$ is divisible by 5 as well.

Therefore $10^{2n}-1$ is also divisible by 5, so $10^{2n}=1+5k$ where k is an integer. Then we can see that $10^{2n}-1=(1+5k)-1=5k$ and therefore $10^{2n}=5k$ or $k=\frac{1}{2}\cdot10^{2n}$ which is clearly less than 10. So the only possible value of $k$ is $\boxed{\frac{1}{2}}$. 

EDIT: Fixed typo. Sorry! :blush:



I'm not sure what you're asking, but here's my sol

In [ ]:
# -*- coding: utf-8 -*-
# GPRO_3b_ec2.py

# Installation instructions for EC2 (run these in your environment before executing the script):
# pip install -U trl peft math_verify transformers datasets huggingface_hub accelerate torch
# Ensure CUDA is available and compatible with your torch version if using GPU.

import os
import re
import shutil # For managing directories if needed

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
from huggingface_hub import login
from peft import LoraConfig, get_peft_model, PeftModel
from math_verify import LatexExtractionConfig, parse, verify
from trl import GRPOConfig, GRPOTrainer

# --- Configuration ---
# Hugging Face Login (recommended: set HUGGING_FACE_HUB_TOKEN environment variable)
try:
    login_token = os.environ.get("HUGGING_FACE_HUB_TOKEN")
    if login_token:
        login(token=login_token, add_to_git_credential=False) # add_to_git_credential=False for non-interactive
        print("Hugging Face Hub login successful using HUGGING_FACE_HUB_TOKEN.")
    else:
        print("HUGGING_FACE_HUB_TOKEN not found. Attempting interactive login if needed by model download.")
        # login() # Or just let operations that need it fail/prompt.
except Exception as e:
    print(f"Hugging Face Hub login attempt issue. Ensure you are logged in if private models are used. Error: {e}")

# Dataset and Model IDs
dataset_id = "AI-MO/NuminaMath-TIR"
model_id = "Qwen/Qwen2-0.5B-Instruct" # Base model for fine-tuning

# Output directory for the trained model
TRAINED_MODEL_DIR = "Qwen2-0.5B-GRPO-NuminaMath-Finetuned"

# System prompt for the task
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)

# --- Dataset Preparation ---
print(f"Loading dataset: {dataset_id}")
# Using a smaller portion for demonstration. Adjust or remove slicing for full training.
# E.g., split=["train", "test"] for full dataset.
# For EC2, manage data size based on instance capabilities and time.
train_dataset_raw = load_dataset(dataset_id, split="train[:1%]") # Use a small slice for faster run
# test_dataset_raw = load_dataset(dataset_id, split="test[:1%]") # Test set not directly used by GRPOTrainer.train, but good for eval

print(f"Loaded raw train dataset: {train_dataset_raw}")

def make_conversation_and_prepare_columns(example):
    """Formats the problem into a chat prompt and ensures 'solution' is available."""
    return {
        "prompt": [ # This will be the main input for the GRPOTrainer
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["problem"]},
        ],
        "solution": example["solution"] # Keep solution for the accuracy reward function
    }

train_dataset = train_dataset_raw.map(make_conversation_and_prepare_columns)

# Remove original columns that are now incorporated into 'prompt' or are not needed
# 'problem' is in 'prompt', 'solution' is kept, 'messages' (original format) is not needed.
train_dataset = train_dataset.remove_columns(["problem", "messages"])

print(f"Processed train dataset: {train_dataset}")
print(f"Train dataset features: {train_dataset.features}")
if len(train_dataset) == 0:
    raise ValueError("Train dataset is empty. Check data loading and slicing.")


# --- Model Loading and PEFT Setup ---
print(f"Loading base model: {model_id}")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",  # Automatically selects precision (bfloat16, float16, or float32)
    device_map="auto",   # Automatically distributes model across available devices (GPU/CPU)
)

# LoRA Configuration
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"], # Modules to apply LoRA to. Check model architecture if changing.
)

model = get_peft_model(model, lora_config)
print("Trainable parameters after PEFT setup:")
model.print_trainable_parameters()

# --- Reward Functions ---
def format_reward(completions, **kwargs):
    """Reward function that checks if the completion has the specified <think>...</think><answer>...</answer> format."""
    pattern = r"^<think>.*?</think>\s*<answer>.*?</answer>$"
    # Completions is a list of lists of dicts: [[{"role": "assistant", "content": "..."}], ...]
    # We need the content string.
    completion_contents = [comp[0]["content"] for comp in completions if comp and comp[0] and "content" in comp[0]]
    rewards_list = [1.0 if re.match(pattern, content, re.DOTALL) else 0.0 for content in completion_contents]
    return rewards_list

def accuracy_reward(completions, **kwargs):
    """Reward function that checks if the answer in the completion matches the ground truth solution."""
    # 'solution' column from the dataset is passed via kwargs by GRPOTrainer if remove_unused_columns=False
    solutions = kwargs["solution"] # This will be a list of solutions, one for each prompt in the batch

    # Completions is List[List[Dict[str, str]]], where outer list corresponds to prompts,
    # inner list corresponds to `num_generations` per prompt.
    # We need to align generated completions with their original solutions.
    # GRPOTrainer calls reward_funcs with `completions` shaped (batch_size * num_generations, 1, dict)
    # and `solution` will be of length `batch_size`. We need to repeat/align `solution`.
    # The GRPOTrainer's documentation/source should clarify how kwargs are batched.
    # Assuming `solution` corresponds to the prompts, and we have `num_generations` for each.
    # Let's assume for now `completions` and `solutions` are aligned by GRPOTrainer or this needs adjustment.
    # The provided example implies completions and solutions are already aligned or broadcasted.
    # `completions` is (batch_size * num_generations), `solutions` is (batch_size)
    # Let `num_generations` be from `training_args`.

    num_gens_per_prompt = kwargs.get("num_generations", 1) # Get from training_args if possible or infer
    if "num_generations" not in kwargs and trainer_args: # Access from global if defined
        num_gens_per_prompt = trainer_args.num_generations

    expanded_solutions = []
    for sol in solutions:
        expanded_solutions.extend([sol] * num_gens_per_prompt)

    completion_contents = [comp[0]["content"] for comp in completions if comp and comp[0] and "content" in comp[0]]
    rewards = []

    for content, solution_text in zip(completion_contents, expanded_solutions):
        gold_parsed = parse(solution_text, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
        # Extract from <answer> tag in generated content
        answer_match = re.search(r"<answer>(.*?)</answer>", content, re.DOTALL)
        if answer_match:
            answer_text = answer_match.group(1).strip()
            answer_parsed = parse(answer_text, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
            if len(gold_parsed) != 0 and len(answer_parsed) != 0:
                try:
                    rewards.append(float(verify(answer_parsed, gold_parsed)))
                except Exception:
                    rewards.append(0.0) # Error during verification
            elif len(gold_parsed) == 0 and len(answer_parsed) == 0 : # Both empty (e.g. non-math textual answer)
                 rewards.append(1.0 if answer_text == solution_text else 0.0) # Simple string match for non-parsable
            elif len(gold_parsed) == 0 and len(answer_parsed) != 0 : # Gold is empty, answer is not
                rewards.append(0.0)
            else: # Gold is not empty, answer is empty or unparsable
                rewards.append(0.0)
        else:
            rewards.append(0.0) # No <answer> tag
    return rewards

# --- Training Arguments and Trainer ---
print("Configuring training arguments...")
# Conditional mixed precision based on hardware support
use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
use_fp16 = torch.cuda.is_available() and not use_bf16

# Global placeholder for training_args to be accessible by reward function if needed for num_generations
trainer_args = None

training_args = GRPOConfig(
    output_dir=TRAINED_MODEL_DIR,
    learning_rate=1e-5,       # Typical learning rate for fine-tuning
    remove_unused_columns=False, # Crucial for accessing 'solution' in accuracy_reward
    gradient_accumulation_steps=4, # Adjust based on VRAM. Original: 16. Reduced for wider compatibility.
    num_train_epochs=1,        # Number of training epochs
    bf16=use_bf16,             # Use bfloat16 if supported
    fp16=use_fp16,             # Use float16 if bfloat16 not supported but CUDA is available

    # per_device_train_batch_size: From TrainingArguments, defaults to 8.
    # This is the number of prompts processed for loss computation by one device.
    per_device_train_batch_size=2, # Adjust based on VRAM. Original: 8 (default). Reduced.

    # batch_size (for GRPO generation step): From GRPOConfig, defaults to 1.
    # Number of prompts to generate completions for simultaneously.
    # batch_size = 1, # Default in GRPOConfig

    # Parameters controlling data preprocessing and generation during training
    max_completion_length=128, # Max length of generated completions. Original: 64. Increased for math.
    num_generations=2,         # Number of completions to generate per prompt. Original: 4. Reduced.
    max_prompt_length=256,     # Max length of prompts. Original: 128. Increased.

    # Reporting and saving
    report_to="tensorboard",   # For EC2, consider "none" or ensure TensorBoard is set up.
    logging_steps=10,
    save_strategy="epoch",     # Save at the end of each epoch. Original: "steps"
    # save_steps=10,           # Used if save_strategy="steps". Original: 10
    # save_total_limit=1,      # Optional: limit the number of saved checkpoints
    push_to_hub=False,         # Set to True to push to Hugging Face Hub
    # hub_model_id="your-username/your-model-name", # Required if push_to_hub=True
)
trainer_args = training_args # Make it accessible globally for reward function

tokenizer_for_training = AutoTokenizer.from_pretrained(model_id)
if tokenizer_for_training.pad_token_id is None:
    tokenizer_for_training.pad_token_id = tokenizer_for_training.eos_token_id
    model.config.pad_token_id = model.config.eos_token_id # Ensure model config also updated

trainer = GRPOTrainer(
    model=model,
    tokenizer=tokenizer_for_training, # Pass tokenizer to GRPOTrainer
    reward_funcs=[format_reward, accuracy_reward],
    args=training_args,
    train_dataset=train_dataset,
)

print("Starting training...")
trainer.train()
print("Training finished.")

# --- Save Model ---
print(f"Saving fine-tuned model (adapter) and tokenizer to {TRAINED_MODEL_DIR}...")
trainer.save_model(TRAINED_MODEL_DIR) # Saves adapter & tokenizer
# tokenizer_for_training.save_pretrained(TRAINED_MODEL_DIR) # Trainer should do this
print(f"Model and tokenizer saved to {TRAINED_MODEL_DIR}.")

# Optional: Zip the model directory if needed for transfer
# shutil.make_archive(f"{TRAINED_MODEL_DIR}_archive", "zip", TRAINED_MODEL_DIR)
# print(f"Model archived to {TRAINED_MODEL_DIR}_archive.zip")


# --- Inference with the Fine-tuned Model ---
print("\n--- Starting Inference with the Fine-tuned Model ---")

# Load the fine-tuned model (base model + adapter) and tokenizer
# AutoModelForCausalLM.from_pretrained on a PEFT-saved directory
# should load the base model and apply the adapter.
print(f"Loading fine-tuned model from {TRAINED_MODEL_DIR} for inference...")
try:
    inference_model = AutoModelForCausalLM.from_pretrained(
        TRAINED_MODEL_DIR,
        torch_dtype="auto", # Use appropriate dtype
        device_map="auto",  # Load on available device
    )
    # Tokenizer should have been saved by trainer.save_model()
    inference_tokenizer = AutoTokenizer.from_pretrained(TRAINED_MODEL_DIR)
    print("Successfully loaded fine-tuned model and tokenizer for inference.")
except Exception as e:
    print(f"Error loading model directly from {TRAINED_MODEL_DIR}: {e}")
    print("Attempting fallback: loading base model and then attaching adapter...")
    base_model_for_inference = AutoModelForCausalLM.from_pretrained(
        model_id, # Base model ID
        torch_dtype="auto",
        device_map="auto",
    )
    inference_model = PeftModel.from_pretrained(base_model_for_inference, TRAINED_MODEL_DIR)
    # Optional: merge LoRA weights for potentially faster inference if not done automatically.
    # This replaces the model with the merged version.
    # inference_model = inference_model.merge_and_unload()
    inference_tokenizer = AutoTokenizer.from_pretrained(model_id) # Fallback to base tokenizer
    print("Fallback loading complete.")

# Ensure pad_token_id is set for the inference tokenizer
if inference_tokenizer.pad_token_id is None:
    inference_tokenizer.pad_token_id = inference_tokenizer.eos_token_id

# Determine device for pipeline (GPU if available, else CPU)
pipeline_device = 0 if torch.cuda.is_available() else -1

print(f"Creating text-generation pipeline on device: {'cuda:0' if pipeline_device == 0 else 'cpu'}")
gen_pipeline = pipeline(
    'text-generation',
    model=inference_model,
    tokenizer=inference_tokenizer,
    device=pipeline_device
)

# Test prompts
prompts_for_testing = [
    "Explain Group Relative Policy Optimization in simple terms.",
    "In 1988, a person's age was equal to the sum of the digits of their birth year. How old was this person?"
]

for user_prompt_text in prompts_for_testing:
    print(f"\nUser Prompt: {user_prompt_text}")

    # Format the prompt using the chat template, including the system prompt used during training
    messages_for_inference = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt_text},
    ]

    if inference_tokenizer.chat_template:
        formatted_model_input = inference_tokenizer.apply_chat_template(
            messages_for_inference,
            tokenize=False,
            add_generation_prompt=True  # Crucial for instruct models to know it's their turn
        )
    else:
        # Fallback if no chat template (should not happen for Qwen2-Instruct)
        print("Warning: Tokenizer does not have a chat template. Using basic formatting.")
        formatted_model_input = f"{SYSTEM_PROMPT}\nUser: {user_prompt_text}\nAssistant:"

    print(f"--- Formatted Input to Model ---\n{formatted_model_input}\n-------------------------------")

    # Generate text
    outputs = gen_pipeline(
        formatted_model_input,
        max_new_tokens=512,  # Max tokens for the generated response
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        #eos_token_id=inference_tokenizer.eos_token_id, # Pipeline usually handles this
        #pad_token_id=inference_tokenizer.pad_token_id # Pipeline usually handles this
    )

    generated_full_text = outputs[0]['generated_text']

    # Extract only the newly generated assistant's reply
    # The `formatted_model_input` is what we sent.
    # The `generated_full_text` contains this input + the model's generation.
    assistant_reply = generated_full_text
    if generated_full_text.startswith(formatted_model_input):
        assistant_reply = generated_full_text[len(formatted_model_input):].strip()
    else:
        # If the pipeline behaves differently or input string had subtle changes (e.g. whitespace)
        # try to find the last part based on assistant markers if possible.
        # For Qwen2, this would be after "<|im_start|>assistant\n"
        # This is a heuristic if the direct prefix strip fails.
        assistant_marker = None
        if "qwen2" in inference_tokenizer.name_or_path.lower():
             assistant_marker = "<|im_start|>assistant" # Don't include \n as it might vary slightly

        if assistant_marker:
            last_occurrence_idx = generated_full_text.rfind(assistant_marker)
            if last_occurrence_idx != -1:
                # Find the end of the marker (e.g., after the newline)
                end_of_marker_idx = generated_full_text.find("\n", last_occurrence_idx)
                if end_of_marker_idx != -1:
                     assistant_reply = generated_full_text[end_of_marker_idx+1:].strip()

    # Clean up trailing special tokens like <|im_end|>
    if assistant_reply.endswith(inference_tokenizer.eos_token) or \
       (hasattr(inference_tokenizer, 'special_tokens_map') and \
        'im_end_token' in inference_tokenizer.special_tokens_map and \
        assistant_reply.endswith(inference_tokenizer.special_tokens_map['im_end_token'])):

        if assistant_reply.endswith(inference_tokenizer.eos_token):
            assistant_reply = assistant_reply[:-len(inference_tokenizer.eos_token)].strip()

        if hasattr(inference_tokenizer, 'special_tokens_map') and \
           'im_end_token' in inference_tokenizer.special_tokens_map and \
           assistant_reply.endswith(inference_tokenizer.special_tokens_map['im_end_token']):
           assistant_reply = assistant_reply[:-len(inference_tokenizer.special_tokens_map['im_end_token'])].strip()


    print(f"--- Assistant's Reply ---\n{assistant_reply}\n-------------------------")

print("\nScript finished.")

ModuleNotFoundError: No module named 'math_verify'

In [ ]:
# -*- coding: utf-8 -*-
# GPRO_3b_ec2.py

# Installation instructions for EC2 (run these in your environment before executing the script):
# pip install -U trl peft math_verify transformers datasets huggingface_hub accelerate torch
# Ensure CUDA is available and compatible with your torch version if using GPU.

import os
import re
import shutil # For managing directories if needed

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
from huggingface_hub import login
from peft import LoraConfig, get_peft_model, PeftModel
from math_verify import LatexExtractionConfig, parse, verify
from trl import GRPOConfig, GRPOTrainer

# --- Configuration ---
# Hugging Face Login (recommended: set HUGGING_FACE_HUB_TOKEN environment variable)
try:
    login_token = os.environ.get("HUGGING_FACE_HUB_TOKEN")
    if login_token:
        login(token=login_token, add_to_git_credential=False) # add_to_git_credential=False for non-interactive
        print("Hugging Face Hub login successful using HUGGING_FACE_HUB_TOKEN.")
    else:
        print("HUGGING_FACE_HUB_TOKEN not found. Attempting interactive login if needed by model download.")
        # login() # Or just let operations that need it fail/prompt.
except Exception as e:
    print(f"Hugging Face Hub login attempt issue. Ensure you are logged in if private models are used. Error: {e}")

# Dataset and Model IDs
dataset_id = "AI-MO/NuminaMath-TIR"
model_id = "Qwen/Qwen2-0.5B-Instruct" # Base model for fine-tuning

# Output directory for the trained model
TRAINED_MODEL_DIR = "Qwen2-0.5B-GRPO-NuminaMath-Finetuned"

# System prompt for the task
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)

# --- Dataset Preparation ---
print(f"Loading dataset: {dataset_id}")
# Using a smaller portion for demonstration. Adjust or remove slicing for full training.
# E.g., split=["train", "test"] for full dataset.
# For EC2, manage data size based on instance capabilities and time.
train_dataset_raw = load_dataset(dataset_id, split="train[:1%]") # Use a small slice for faster run
# test_dataset_raw = load_dataset(dataset_id, split="test[:1%]") # Test set not directly used by GRPOTrainer.train, but good for eval

print(f"Loaded raw train dataset: {train_dataset_raw}")

def make_conversation_and_prepare_columns(example):
    """Formats the problem into a chat prompt and ensures 'solution' is available."""
    return {
        "prompt": [ # This will be the main input for the GRPOTrainer
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["problem"]},
        ],
        "solution": example["solution"] # Keep solution for the accuracy reward function
    }

train_dataset = train_dataset_raw.map(make_conversation_and_prepare_columns)

# Remove original columns that are now incorporated into 'prompt' or are not needed
# 'problem' is in 'prompt', 'solution' is kept, 'messages' (original format) is not needed.
train_dataset = train_dataset.remove_columns(["problem", "messages"])

print(f"Processed train dataset: {train_dataset}")
print(f"Train dataset features: {train_dataset.features}")
if len(train_dataset) == 0:
    raise ValueError("Train dataset is empty. Check data loading and slicing.")


# --- Model Loading and PEFT Setup ---
print(f"Loading base model: {model_id}")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",  # Automatically selects precision (bfloat16, float16, or float32)
    device_map="auto",   # Automatically distributes model across available devices (GPU/CPU)
)

# LoRA Configuration
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"], # Modules to apply LoRA to. Check model architecture if changing.
)

model = get_peft_model(model, lora_config)
print("Trainable parameters after PEFT setup:")
model.print_trainable_parameters()

# --- Reward Functions ---
def format_reward(completions, **kwargs):
    """Reward function that checks if the completion has the specified <think>...</think><answer>...</answer> format."""
    pattern = r"^<think>.*?</think>\s*<answer>.*?</answer>$"
    # Completions is a list of lists of dicts: [[{"role": "assistant", "content": "..."}], ...]
    # We need the content string.
    completion_contents = [comp[0]["content"] for comp in completions if comp and comp[0] and "content" in comp[0]]
    rewards_list = [1.0 if re.match(pattern, content, re.DOTALL) else 0.0 for content in completion_contents]
    return rewards_list

def accuracy_reward(completions, **kwargs):
    """Reward function that checks if the answer in the completion matches the ground truth solution."""
    # 'solution' column from the dataset is passed via kwargs by GRPOTrainer if remove_unused_columns=False
    solutions = kwargs["solution"] # This will be a list of solutions, one for each prompt in the batch

    # Completions is List[List[Dict[str, str]]], where outer list corresponds to prompts,
    # inner list corresponds to `num_generations` per prompt.
    # We need to align generated completions with their original solutions.
    # GRPOTrainer calls reward_funcs with `completions` shaped (batch_size * num_generations, 1, dict)
    # and `solution` will be of length `batch_size`. We need to repeat/align `solution`.
    # The GRPOTrainer's documentation/source should clarify how kwargs are batched.
    # Assuming `solution` corresponds to the prompts, and we have `num_generations` for each.
    # Let's assume for now `completions` and `solutions` are aligned by GRPOTrainer or this needs adjustment.
    # The provided example implies completions and solutions are already aligned or broadcasted.
    # `completions` is (batch_size * num_generations), `solutions` is (batch_size)
    # Let `num_generations` be from `training_args`.

    num_gens_per_prompt = kwargs.get("num_generations", 1) # Get from training_args if possible or infer
    if "num_generations" not in kwargs and trainer_args: # Access from global if defined
        num_gens_per_prompt = trainer_args.num_generations

    expanded_solutions = []
    for sol in solutions:
        expanded_solutions.extend([sol] * num_gens_per_prompt)

    completion_contents = [comp[0]["content"] for comp in completions if comp and comp[0] and "content" in comp[0]]
    rewards = []

    for content, solution_text in zip(completion_contents, expanded_solutions):
        gold_parsed = parse(solution_text, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
        # Extract from <answer> tag in generated content
        answer_match = re.search(r"<answer>(.*?)</answer>", content, re.DOTALL)
        if answer_match:
            answer_text = answer_match.group(1).strip()
            answer_parsed = parse(answer_text, extraction_mode="first_match", extraction_config=[LatexExtractionConfig()])
            if len(gold_parsed) != 0 and len(answer_parsed) != 0:
                try:
                    rewards.append(float(verify(answer_parsed, gold_parsed)))
                except Exception:
                    rewards.append(0.0) # Error during verification
            elif len(gold_parsed) == 0 and len(answer_parsed) == 0 : # Both empty (e.g. non-math textual answer)
                 rewards.append(1.0 if answer_text == solution_text else 0.0) # Simple string match for non-parsable
            elif len(gold_parsed) == 0 and len(answer_parsed) != 0 : # Gold is empty, answer is not
                rewards.append(0.0)
            else: # Gold is not empty, answer is empty or unparsable
                rewards.append(0.0)
        else:
            rewards.append(0.0) # No <answer> tag
    return rewards

# --- Training Arguments and Trainer ---
print("Configuring training arguments...")
# Conditional mixed precision based on hardware support
use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
use_fp16 = torch.cuda.is_available() and not use_bf16

# Global placeholder for training_args to be accessible by reward function if needed for num_generations
trainer_args = None

training_args = GRPOConfig(
    output_dir=TRAINED_MODEL_DIR,
    learning_rate=1e-5,       # Typical learning rate for fine-tuning
    remove_unused_columns=False, # Crucial for accessing 'solution' in accuracy_reward
    gradient_accumulation_steps=4, # Adjust based on VRAM. Original: 16. Reduced for wider compatibility.
    num_train_epochs=1,        # Number of training epochs
    bf16=use_bf16,             # Use bfloat16 if supported
    fp16=use_fp16,             # Use float16 if bfloat16 not supported but CUDA is available

    # per_device_train_batch_size: From TrainingArguments, defaults to 8.
    # This is the number of prompts processed for loss computation by one device.
    per_device_train_batch_size=2, # Adjust based on VRAM. Original: 8 (default). Reduced.

    # batch_size (for GRPO generation step): From GRPOConfig, defaults to 1.
    # Number of prompts to generate completions for simultaneously.
    # batch_size = 1, # Default in GRPOConfig

    # Parameters controlling data preprocessing and generation during training
    max_completion_length=128, # Max length of generated completions. Original: 64. Increased for math.
    num_generations=2,         # Number of completions to generate per prompt. Original: 4. Reduced.
    max_prompt_length=256,     # Max length of prompts. Original: 128. Increased.

    # Reporting and saving
    report_to="tensorboard",   # For EC2, consider "none" or ensure TensorBoard is set up.
    logging_steps=10,
    save_strategy="epoch",     # Save at the end of each epoch. Original: "steps"
    # save_steps=10,           # Used if save_strategy="steps". Original: 10
    # save_total_limit=1,      # Optional: limit the number of saved checkpoints
    push_to_hub=False,         # Set to True to push to Hugging Face Hub
    # hub_model_id="your-username/your-model-name", # Required if push_to_hub=True
)
trainer_args = training_args # Make it accessible globally for reward function

tokenizer_for_training = AutoTokenizer.from_pretrained(model_id)
if tokenizer_for_training.pad_token_id is None:
    tokenizer_for_training.pad_token_id = tokenizer_for_training.eos_token_id
    model.config.pad_token_id = model.config.eos_token_id # Ensure model config also updated

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer_for_training, # Pass tokenizer to GRPOTrainer
    reward_funcs=[format_reward, accuracy_reward],
    args=training_args,
    train_dataset=train_dataset,
)

print("Starting training...")
trainer.train()
print("Training finished.")

# --- Save Model ---
print(f"Saving fine-tuned model (adapter) and tokenizer to {TRAINED_MODEL_DIR}...")
trainer.save_model(TRAINED_MODEL_DIR) # Saves adapter & tokenizer
# tokenizer_for_training.save_pretrained(TRAINED_MODEL_DIR) # Trainer should do this
print(f"Model and tokenizer saved to {TRAINED_MODEL_DIR}.")

# Optional: Zip the model directory if needed for transfer
# shutil.make_archive(f"{TRAINED_MODEL_DIR}_archive", "zip", TRAINED_MODEL_DIR)
# print(f"Model archived to {TRAINED_MODEL_DIR}_archive.zip")


# --- Inference with the Fine-tuned Model ---
print("\n--- Starting Inference with the Fine-tuned Model ---")

# Load the fine-tuned model (base model + adapter) and tokenizer
# AutoModelForCausalLM.from_pretrained on a PEFT-saved directory
# should load the base model and apply the adapter.
print(f"Loading fine-tuned model from {TRAINED_MODEL_DIR} for inference...")
try:
    inference_model = AutoModelForCausalLM.from_pretrained(
        TRAINED_MODEL_DIR,
        torch_dtype="auto", # Use appropriate dtype
        device_map="auto",  # Load on available device
    )
    # Tokenizer should have been saved by trainer.save_model()
    inference_tokenizer = AutoTokenizer.from_pretrained(TRAINED_MODEL_DIR)
    print("Successfully loaded fine-tuned model and tokenizer for inference.")
except Exception as e:
    print(f"Error loading model directly from {TRAINED_MODEL_DIR}: {e}")
    print("Attempting fallback: loading base model and then attaching adapter...")
    base_model_for_inference = AutoModelForCausalLM.from_pretrained(
        model_id, # Base model ID
        torch_dtype="auto",
        device_map="auto",
    )
    inference_model = PeftModel.from_pretrained(base_model_for_inference, TRAINED_MODEL_DIR)
    # Optional: merge LoRA weights for potentially faster inference if not done automatically.
    # This replaces the model with the merged version.
    # inference_model = inference_model.merge_and_unload()
    inference_tokenizer = AutoTokenizer.from_pretrained(model_id) # Fallback to base tokenizer
    print("Fallback loading complete.")

# Ensure pad_token_id is set for the inference tokenizer
if inference_tokenizer.pad_token_id is None:
    inference_tokenizer.pad_token_id = inference_tokenizer.eos_token_id

# Determine device for pipeline (GPU if available, else CPU)
pipeline_device = 0 if torch.cuda.is_available() else -1

print(f"Creating text-generation pipeline on device: {'cuda:0' if pipeline_device == 0 else 'cpu'}")
gen_pipeline = pipeline(
    'text-generation',
    model=inference_model,
    tokenizer=inference_tokenizer,
    device=pipeline_device
)

# Test prompts
prompts_for_testing = [
    "Explain Group Relative Policy Optimization in simple terms.",
    "In 1988, a person's age was equal to the sum of the digits of their birth year. How old was this person?"
]

for user_prompt_text in prompts_for_testing:
    print(f"\nUser Prompt: {user_prompt_text}")

    # Format the prompt using the chat template, including the system prompt used during training
    messages_for_inference = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt_text},
    ]

    if inference_tokenizer.chat_template:
        formatted_model_input = inference_tokenizer.apply_chat_template(
            messages_for_inference,
            tokenize=False,
            add_generation_prompt=True  # Crucial for instruct models to know it's their turn
        )
    else:
        # Fallback if no chat template (should not happen for Qwen2-Instruct)
        print("Warning: Tokenizer does not have a chat template. Using basic formatting.")
        formatted_model_input = f"{SYSTEM_PROMPT}\nUser: {user_prompt_text}\nAssistant:"

    print(f"--- Formatted Input to Model ---\n{formatted_model_input}\n-------------------------------")

    # Generate text
    outputs = gen_pipeline(
        formatted_model_input,
        max_new_tokens=512,  # Max tokens for the generated response
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        #eos_token_id=inference_tokenizer.eos_token_id, # Pipeline usually handles this
        #pad_token_id=inference_tokenizer.pad_token_id # Pipeline usually handles this
    )

    generated_full_text = outputs[0]['generated_text']

    # Extract only the newly generated assistant's reply
    # The `formatted_model_input` is what we sent.
    # The `generated_full_text` contains this input + the model's generation.
    assistant_reply = generated_full_text
    if generated_full_text.startswith(formatted_model_input):
        assistant_reply = generated_full_text[len(formatted_model_input):].strip()
    else:
        # If the pipeline behaves differently or input string had subtle changes (e.g. whitespace)
        # try to find the last part based on assistant markers if possible.
        # For Qwen2, this would be after "<|im_start|>assistant\n"
        # This is a heuristic if the direct prefix strip fails.
        assistant_marker = None
        if "qwen2" in inference_tokenizer.name_or_path.lower():
             assistant_marker = "<|im_start|>assistant" # Don't include \n as it might vary slightly

        if assistant_marker:
            last_occurrence_idx = generated_full_text.rfind(assistant_marker)
            if last_occurrence_idx != -1:
                # Find the end of the marker (e.g., after the newline)
                end_of_marker_idx = generated_full_text.find("\n", last_occurrence_idx)
                if end_of_marker_idx != -1:
                     assistant_reply = generated_full_text[end_of_marker_idx+1:].strip()

    # Clean up trailing special tokens like <|im_end|>
    if assistant_reply.endswith(inference_tokenizer.eos_token) or \
       (hasattr(inference_tokenizer, 'special_tokens_map') and \
        'im_end_token' in inference_tokenizer.special_tokens_map and \
        assistant_reply.endswith(inference_tokenizer.special_tokens_map['im_end_token'])):

        if assistant_reply.endswith(inference_tokenizer.eos_token):
            assistant_reply = assistant_reply[:-len(inference_tokenizer.eos_token)].strip()

        if hasattr(inference_tokenizer, 'special_tokens_map') and \
           'im_end_token' in inference_tokenizer.special_tokens_map and \
           assistant_reply.endswith(inference_tokenizer.special_tokens_map['im_end_token']):
           assistant_reply = assistant_reply[:-len(inference_tokenizer.special_tokens_map['im_end_token'])].strip()


    print(f"--- Assistant's Reply ---\n{assistant_reply}\n-------------------------")

print("\nScript finished.")

HUGGING_FACE_HUB_TOKEN not found. Attempting interactive login if needed by model download.
Loading dataset: AI-MO/NuminaMath-TIR
Loaded raw train dataset: Dataset({
    features: ['problem', 'solution', 'messages'],
    num_rows: 724
})
Processed train dataset: Dataset({
    features: ['solution', 'prompt'],
    num_rows: 724
})
Train dataset features: {'solution': Value(dtype='string', id=None), 'prompt': [{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}]}
Loading base model: Qwen/Qwen2-0.5B-Instruct
Trainable parameters after PEFT setup:
trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093
Configuring training arguments...


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# %%capture
# !pip install transformers torch accelerate sentencepiece

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import re
import random

# --- Model Setup ---
MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat" # Using a 0.5B Qwen Chat model

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype="auto", # Use bfloat16 if available, otherwise float16
        device_map="auto"   # Automatically use GPU if available
    )
    print(f"Successfully loaded model and tokenizer: {MODEL_NAME}")
    if torch.cuda.is_available():
        print(f"Model is on GPU: {model.device}")
    else:
        print("Model is on CPU. Execution might be slow.")

except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have internet access and the model name is correct.")
    print("If you are repeatedly facing issues, try restarting the Colab runtime.")
    model = None # Set model to None if loading fails


# --- DSL Definition and Parser ---
# Operands: A[r,c], B[r,c], M<id>
# C[r,c] = <operand> OR M<id> = <operand1> <op> <operand2>

# Regex for parsing DSL lines
# M0 = A[0,0] + B[0,0]  OR  M1 = M0 * A[1,1] OR C[0,0] = M1
ASSIGN_INTERMEDIATE_REGEX = re.compile(r"^\s*(M\d+)\s*=\s*([A-Z]\[\d,\d\]|M\d+)\s*([+\-*])\s*([A-Z]\[\d,\d\]|M\d+)\s*$")
ASSIGN_OUTPUT_REGEX = re.compile(r"^\s*(C\[\d,\d\])\s*=\s*([A-Z]\[\d,\d\]|M\d+)\s*$")
ASSIGN_INTERMEDIATE_SINGLE_REGEX = re.compile(r"^\s*(M\d+)\s*=\s*([A-Z]\[\d,\d\]|M\d+)\s*$") # M0 = A[0,0]

def parse_dsl_step(line):
    """Parses a single DSL step string."""
    match_inter = ASSIGN_INTERMEDIATE_REGEX.match(line)
    if match_inter:
        return {"type": "intermediate_op", "out": match_inter.group(1), "op1": match_inter.group(2),
                "op_type": match_inter.group(3), "op2": match_inter.group(4)}

    match_output = ASSIGN_OUTPUT_REGEX.match(line)
    if match_output:
        return {"type": "output_assign", "out_cell": match_output.group(1), "source": match_output.group(2)}

    match_inter_single = ASSIGN_INTERMEDIATE_SINGLE_REGEX.match(line)
    if match_inter_single: # For M0 = A[0,0] type assignments
        return {"type": "intermediate_assign_single", "out": match_inter_single.group(1), "source": match_inter_single.group(2)}

    return None # Parse error

# --- Algorithm Executor ---
class MatrixMathState:
    def __init__(self, a, b):
        if not (a.shape == (2,2) and b.shape == (2,2)):
            raise ValueError("Matrices A and B must be 2x2.")
        self.matrix_a = a
        self.matrix_b = b
        self.expected_c = np.matmul(a, b)
        self.intermediate_vars = {} # Store M0, M1, ...
        self.output_c = np.full((2,2), np.nan) # Initialize with NaNs
        self.errors = []
        self.dsl_steps_executed = []

    def get_operand_value(self, operand_str):
        if operand_str.startswith("M"):
            if operand_str in self.intermediate_vars:
                return self.intermediate_vars[operand_str]
            else:
                self.errors.append(f"Error: Intermediate variable {operand_str} not found.")
                return None
        elif operand_str.startswith("A["):
            try:
                r, c = map(int, operand_str[2:-1].split(','))
                return self.matrix_a[r,c]
            except:
                self.errors.append(f"Error: Invalid format or index for matrix A operand: {operand_str}")
                return None
        elif operand_str.startswith("B["):
            try:
                r, c = map(int, operand_str[2:-1].split(','))
                return self.matrix_b[r,c]
            except:
                self.errors.append(f"Error: Invalid format or index for matrix B operand: {operand_str}")
                return None
        self.errors.append(f"Error: Unknown operand type: {operand_str}")
        return None

    def execute_step(self, parsed_step, original_line=""):
        if parsed_step is None:
            self.errors.append(f"Parse Error for line: {original_line}")
            return False

        op_type = parsed_step["type"]
        success = False

        if op_type == "intermediate_op":
            val1 = self.get_operand_value(parsed_step["op1"])
            val2 = self.get_operand_value(parsed_step["op2"])
            if val1 is not None and val2 is not None:
                op = parsed_step["op_type"]
                if op == '+': result = val1 + val2
                elif op == '-': result = val1 - val2
                elif op == '*': result = val1 * val2
                else:
                    self.errors.append(f"Unknown arithmetic operator: {op}")
                    return False
                self.intermediate_vars[parsed_step["out"]] = result
                success = True
        elif op_type == "intermediate_assign_single":
            val_source = self.get_operand_value(parsed_step["source"])
            if val_source is not None:
                self.intermediate_vars[parsed_step["out"]] = val_source
                success = True
        elif op_type == "output_assign":
            val_source = self.get_operand_value(parsed_step["source"])
            if val_source is not None:
                try:
                    cell_str = parsed_step["out_cell"] # C[r,c]
                    r, c = map(int, cell_str[2:-1].split(','))
                    if 0 <= r < 2 and 0 <= c < 2:
                        self.output_c[r,c] = val_source
                        success = True
                    else:
                        self.errors.append(f"Error: Output cell C[{r},{c}] out of bounds.")
                except Exception as e:
                    self.errors.append(f"Error parsing output cell {parsed_step['out_cell']}: {e}")
        else:
            self.errors.append(f"Unknown parsed step type: {op_type}")

        if success:
            self.dsl_steps_executed.append(original_line if original_line else str(parsed_step))
        return success

    def is_c_complete(self):
        return not np.isnan(self.output_c).any()

    def verify_c(self):
        if not self.is_c_complete():
            return False
        return np.allclose(self.output_c, self.expected_c)

# --- LLM Interaction Logic ---
def get_llm_step_suggestion(current_prompt_messages):
    if model is None or tokenizer is None:
        return "Error: Model not loaded."

    # The Qwen1.5-Chat model expects a list of messages
    input_ids = tokenizer.apply_chat_template(
        current_prompt_messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=50, # Max length of the DSL step string
        pad_token_id=tokenizer.eos_token_id, # Important for stopping criteria
        do_sample=True, # Enable sampling for more diverse outputs
        temperature=0.7, # Control randomness; lower is more deterministic
        top_k=50,
        top_p=0.95
    )
    response_ids = outputs[0][input_ids.shape[-1]:] # Get only the generated tokens
    llm_output = tokenizer.decode(response_ids, skip_special_tokens=True).strip()
    return llm_output

def generate_matrix_mult_algorithm(matrix_a_str, matrix_b_str, max_llm_steps=20):
    """
    Iteratively prompts the LLM to generate a full algorithm.
    """
    state = MatrixMathState(eval(matrix_a_str), eval(matrix_b_str)) # eval is generally unsafe, but used here for simplicity with np.array strings

    # System message to set the context for the chat model
    system_message = f"""You are an AI assistant helping to generate steps for a 2x2 matrix multiplication: C = A * B.
Your task is to output ONLY ONE valid DSL step at a time to compute the matrix C.
Matrices A and B are 2x2.
Valid DSL Step Formats:
1. Define an intermediate variable M<id> (e.g., M0, M1, M2, ...):
   M<id> = <operand1> + <operand2>
   M<id> = <operand1> - <operand2>
   M<id> = <operand1> * <operand2>
   M<id> = <operand>  (e.g. M0 = A[0,0])
   Operands can be A[r,c], B[r,c], or a previously defined M<id>.
2. Assign to an output cell C[r,c]:
   C[r,c] = <operand> (where operand is an M<id> or A[r,c] or B[r,c])

Example of a valid first step: M0 = A[0,0] * B[0,0]
Another example: M1 = A[0,1] + A[1,1]
Then, for example: C[0,0] = M0 (if M0 was defined as a product for C[0,0])

Do not output any explanations, just the single DSL line.
Focus on correctness and completing all C[0,0], C[0,1], C[1,0], C[1,1].
"""

    # Initialize conversation history with the system message
    conversation_history = [{"role": "system", "content": system_message}]

    for step_num in range(max_llm_steps):
        if state.is_c_complete():
            break

        current_state_summary = f"Current algorithm state:\nMatrices:\nA = {matrix_a_str}\nB = {matrix_b_str}\n"
        current_state_summary += "Executed Steps:\n" + ("\n".join(state.dsl_steps_executed) if state.dsl_steps_executed else "None yet.") + "\n"
        current_state_summary += f"Intermediate Variables (M values): {state.intermediate_vars}\n"
        current_state_summary += f"Output Matrix C (NaN means not yet computed):\n{state.output_c}\n"
        current_state_summary += "Provide the NEXT SINGLE DSL step:"

        # Add user message to conversation
        user_message = {"role": "user", "content": current_state_summary}
        prompt_messages_for_llm = conversation_history + [user_message]

        llm_dsl_line = get_llm_step_suggestion(prompt_messages_for_llm)

        if llm_dsl_line.startswith("Error:"): # Handle model loading error
            state.errors.append(llm_dsl_line)
            break # Stop if model isn't working

        # Add assistant's response (the DSL line) to history for next turn
        assistant_message = {"role": "assistant", "content": llm_dsl_line}
        conversation_history.append(user_message) # Add user's prompt that led to this
        conversation_history.append(assistant_message) # Add assistant's reply


        parsed = parse_dsl_step(llm_dsl_line)

        if parsed:
            if not state.execute_step(parsed, llm_dsl_line):
                # Error already logged in state.execute_step
                # Decide if we want to stop on first execution error, or let LLM try to recover
                # For this test, let's log and continue, LLM might output something different next
                print(f"  LLM Output (Step {step_num+1}): '{llm_dsl_line}' -> Parsed, but execution failed. Error: {state.errors[-1] if state.errors else 'Unknown exec error'}")
            else:
                 print(f"  LLM Output (Step {step_num+1}): '{llm_dsl_line}' -> Executed successfully.")
        else:
            state.errors.append(f"LLM output could not be parsed: '{llm_dsl_line}'")
            print(f"  LLM Output (Step {step_num+1}): '{llm_dsl_line}' -> Parse Error.")
            # Optionally, break if LLM consistently gives unparsable output

    return state

# --- Main Test Loop ---
if model is None:
    print("\nSkipping main test loop as model failed to load.")
else:
    print("\n--- Starting 2x2 Matrix Multiplication Test ---")
    num_tests = 10
    correct_algorithms = 0

    for i in range(num_tests):
        print(f"\n--- Test Case {i+1}/{num_tests} ---")
        # Generate random 2x2 matrices with integer values for simplicity
        # Using np.array representation directly in strings for the prompt
        # This avoids issues with LLM misinterpreting list of lists syntax sometimes
        mat_a_np = np.random.randint(-5, 6, size=(2,2))
        mat_b_np = np.random.randint(-5, 6, size=(2,2))

        # Convert numpy arrays to string representations for the prompt
        # e.g., "np.array([[1, 2], [3, 4]])"
        mat_a_str = f"np.array({mat_a_np.tolist()})"
        mat_b_str = f"np.array({mat_b_np.tolist()})"

        print(f"Matrix A:\n{mat_a_np}")
        print(f"Matrix B:\n{mat_b_np}")
        expected_c = np.matmul(mat_a_np, mat_b_np)
        print(f"Expected C:\n{expected_c}")

        final_state = generate_matrix_mult_algorithm(mat_a_str, mat_b_str)

        print("\nGenerated DSL Algorithm:")
        if final_state.dsl_steps_executed:
            for step_line in final_state.dsl_steps_executed:
                print(step_line)
        else:
            print("No valid DSL steps were executed.")

        if final_state.errors:
            print("\nErrors encountered during generation/execution:")
            for err in final_state.errors:
                print(f"- {err}")

        print(f"\nFinal Computed C (by LLM algorithm):\n{final_state.output_c}")

        if final_state.is_c_complete():
            if final_state.verify_c():
                print("Result: CORRECT")
                correct_algorithms += 1
            else:
                print("Result: INCORRECT (C is complete but values are wrong)")
        else:
            print("Result: INCOMPLETE (C was not fully computed)")

    print(f"\n--- Test Summary ---")
    print(f"Total correct algorithms: {correct_algorithms}/{num_tests} ({correct_algorithms/num_tests*100:.2f}%)")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Successfully loaded model and tokenizer: Qwen/Qwen1.5-0.5B-Chat
Model is on GPU: cuda:0

--- Starting 2x2 Matrix Multiplication Test ---

--- Test Case 1/10 ---
Matrix A:
[[-1 -1]
 [-4 -4]]
Matrix B:
[[ 2  5]
 [ 1 -4]]
Expected C:
[[ -3  -1]
 [-12  -4]]
  LLM Output (Step 1): 'Sure! Here's the next DSL step to compute the matrix C:
```
C = np.dot(A, B)
```' -> Parse Error.
  LLM Output (Step 2): 'Here's the next DSL step to compute the matrix C:
```
M = np.linalg.norm(C)
C = A @ B
```' -> Parse Error.
  LLM Output (Step 3): 'Here's the next DSL step to compute the matrix C:
```
M = np.linalg.norm(np.dot(A, B))
C = A @ B
```' -> Parse Error.
  LLM Output (Step 4): 'Here's the next DSL step to compute the matrix C:
```
M = np.linalg.norm(np.dot(A, B))
C = A @ B
```' -> Parse Error.
  LLM Output (Step 5): 'Here's the next DSL step to compute the matrix C:
```
M = np.linalg.norm(np.dot(A, B))
C = A @ B
```' -> Parse Error.
  LLM Output (Step 6): 'Here's the next DSL step to compute the mat

In [ ]:
# %%capture
# !pip install transformers torch accelerate sentencepiece

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import re
import random

# --- Model Setup ---
MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat"

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype="auto",
        device_map="auto"
    )
    print(f"Successfully loaded model and tokenizer: {MODEL_NAME}")
    if torch.cuda.is_available():
        print(f"Model is on GPU: {model.device}")
    else:
        print("Model is on CPU. Execution might be slow.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

# --- DSL Definition and Parser ---
ASSIGN_INTERMEDIATE_REGEX = re.compile(r"^\s*(M\d+)\s*=\s*([A-Z]\[\d,\d\]|M\d+)\s*([+\-*])\s*([A-Z]\[\d,\d\]|M\d+)\s*$")
ASSIGN_OUTPUT_REGEX = re.compile(r"^\s*(C\[\d,\d\])\s*=\s*([A-Z]\[\d,\d\]|M\d+)\s*$")
ASSIGN_INTERMEDIATE_SINGLE_REGEX = re.compile(r"^\s*(M\d+)\s*=\s*([A-Z]\[\d,\d\]|M\d+)\s*$")

def parse_dsl_step(line):
    line = line.strip()
    match_inter = ASSIGN_INTERMEDIATE_REGEX.match(line)
    if match_inter:
        return {"type": "intermediate_op", "out": match_inter.group(1), "op1": match_inter.group(2),
                "op_type": match_inter.group(3), "op2": match_inter.group(4)}

    match_output = ASSIGN_OUTPUT_REGEX.match(line)
    if match_output:
        return {"type": "output_assign", "out_cell": match_output.group(1), "source": match_output.group(2)}

    match_inter_single = ASSIGN_INTERMEDIATE_SINGLE_REGEX.match(line)
    if match_inter_single:
        return {"type": "intermediate_assign_single", "out": match_inter_single.group(1), "source": match_inter_single.group(2)}
    return None

# --- Algorithm Executor ---
class MatrixMathState:
    def __init__(self, a, b): # Expects numpy arrays
        if not (isinstance(a, np.ndarray) and isinstance(b, np.ndarray) and a.shape == (2,2) and b.shape == (2,2)):
            raise ValueError("Matrices A and B must be 2x2 numpy arrays.")
        self.matrix_a = a
        self.matrix_b = b
        self.expected_c = np.matmul(a, b)
        self.intermediate_vars = {}
        self.output_c = np.full((2,2), np.nan)
        self.errors = []
        self.dsl_steps_executed = []

    def get_operand_value(self, operand_str):
        operand_str = operand_str.strip()
        if operand_str.startswith("M"):
            if operand_str in self.intermediate_vars:
                return self.intermediate_vars[operand_str]
            else:
                self.errors.append(f"Error: Intermediate variable {operand_str} not found.")
                return None
        elif operand_str.startswith("A["):
            try:
                r, c = map(int, operand_str[2:-1].split(','))
                return self.matrix_a[r,c]
            except:
                self.errors.append(f"Error: Invalid format or index for matrix A operand: {operand_str}")
                return None
        elif operand_str.startswith("B["):
            try:
                r, c = map(int, operand_str[2:-1].split(','))
                return self.matrix_b[r,c]
            except:
                self.errors.append(f"Error: Invalid format or index for matrix B operand: {operand_str}")
                return None
        self.errors.append(f"Error: Unknown operand type: {operand_str}")
        return None

    def execute_step(self, parsed_step, original_line=""):
        if parsed_step is None: # Should not happen if clean_llm_output returns FORMAT_ERROR for unparsable
            self.errors.append(f"Internal Error: execute_step called with None parsed_step for line: {original_line}")
            return False

        op_type = parsed_step["type"]
        success = False
        try:
            if op_type == "intermediate_op":
                val1 = self.get_operand_value(parsed_step["op1"])
                val2 = self.get_operand_value(parsed_step["op2"])
                if val1 is not None and val2 is not None:
                    op = parsed_step["op_type"]
                    if op == '+': result = val1 + val2
                    elif op == '-': result = val1 - val2
                    elif op == '*': result = val1 * val2
                    else:
                        self.errors.append(f"Unknown arithmetic operator: {op}")
                        return False # Keep this return
                    self.intermediate_vars[parsed_step["out"]] = result
                    success = True
            elif op_type == "intermediate_assign_single":
                val_source = self.get_operand_value(parsed_step["source"])
                if val_source is not None:
                    self.intermediate_vars[parsed_step["out"]] = val_source
                    success = True
            elif op_type == "output_assign":
                val_source = self.get_operand_value(parsed_step["source"])
                if val_source is not None:
                    cell_str = parsed_step["out_cell"]
                    r, c = map(int, cell_str[2:-1].split(','))
                    if 0 <= r < 2 and 0 <= c < 2:
                        self.output_c[r,c] = val_source
                        success = True
                    else:
                        self.errors.append(f"Error: Output cell C[{r},{c}] out of bounds.")
            else:
                self.errors.append(f"Unknown parsed step type: {op_type}")
        except Exception as e:
            self.errors.append(f"Execution error for '{original_line}': {e}")
            success = False

        if success:
            self.dsl_steps_executed.append(original_line)
        # If not success, error is already logged by get_operand_value or within this function
        return success

    def is_c_complete(self):
        return not np.isnan(self.output_c).any()

    def verify_c(self):
        if not self.is_c_complete():
            return False
        return np.allclose(self.output_c, self.expected_c)

# --- LLM Interaction Logic ---
def clean_llm_output(raw_output):
    cleaned = raw_output.strip()

    code_block_match = re.search(r"```(?:[a-zA-Z]+\n)?(.*?)```", cleaned, re.DOTALL)
    if code_block_match:
        potential_lines = code_block_match.group(1).strip().split('\n')
        if potential_lines:
            cleaned = potential_lines[0].strip()

    prefixes_to_remove = [
        "Sure! Here's the next DSL step:", "Okay, the next DSL step is:",
        "Here's the next DSL step:", "The next DSL step is:", "Next DSL step:",
        "Here is the next step:", "Here's the step:", "Okay, here's the step:",
        "Next single DSL step:", "Next DSL Step:", "The DSL step is:",
        "The next step is:", "Alright, the next step is:"
    ]
    temp_cleaned_after_prefix_removal = cleaned
    for prefix in prefixes_to_remove:
        if temp_cleaned_after_prefix_removal.lower().startswith(prefix.lower()):
            temp_cleaned_after_prefix_removal = temp_cleaned_after_prefix_removal[len(prefix):].strip()
            break
    cleaned = temp_cleaned_after_prefix_removal

    possible_dsl_lines = cleaned.split('\n')
    for line_candidate in possible_dsl_lines:
        line_candidate = line_candidate.strip()
        if parse_dsl_step(line_candidate):
            return line_candidate

    for line_candidate in possible_dsl_lines:
        if line_candidate.strip():
            return f"FORMAT_ERROR: {line_candidate.strip()}"

    return ""


def get_llm_step_suggestion(current_prompt_messages):
    if model is None or tokenizer is None:
        # This case should be handled before calling, but as a safeguard:
        print("Error: Model or tokenizer not loaded in get_llm_step_suggestion.")
        return "Error: Model not loaded."

    input_ids = tokenizer.apply_chat_template(
        current_prompt_messages,
        add_generation_prompt=True, # This is important for chat models
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=25, # Reduced
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.4, # Reduced
        top_k=30,        # Reduced
        top_p=0.9,
        no_repeat_ngram_size=2 # More aggressive
    )
    response_ids = outputs[0][input_ids.shape[-1]:]
    llm_raw_output = tokenizer.decode(response_ids, skip_special_tokens=True).strip()
    return clean_llm_output(llm_raw_output)


def generate_matrix_mult_algorithm(current_mat_a_np, current_mat_b_np, max_llm_steps=20):
    state = MatrixMathState(current_mat_a_np, current_mat_b_np) # Pass numpy arrays directly

    system_message = f"""ASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B).
NO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE.
If stuck, output ONLY: Error: Cannot determine next step.

DSL FORMAT:
M<id> = <op1> + <op2>
M<id> = <op1> - <op2>
M<id> = <op1> * <op2>
M<id> = <operand>
C[r,c] = <operand>
(Operands: A[r,c], B[r,c], or M<id>)

VALID DSL EXAMPLES:
M0 = A[0,0] * B[0,0]
M1 = M0 + A[0,1]
C[0,0] = M1
"""
    conversation_history = [{"role": "system", "content": system_message}]

    example_matrix_a_repr = "[[1, 2], [3, 4]]"
    example_matrix_b_repr = "[[5, 6], [7, 8]]"
    example_output_c_repr = "[[nan, nan],\n [nan, nan]]" # Explicitly two lines for example
    example_initial_user_prompt = (
        f"Current algorithm state:\nMatrices:\nA = {example_matrix_a_repr}\nB = {example_matrix_b_repr}\n"
        "Executed Steps:\nNone yet.\n"
        "Intermediate Variables (M values): {}\n"
        f"Output Matrix C (NaN means not yet computed):\n{example_output_c_repr}\n"
        "Provide the NEXT SINGLE DSL step according to the rules:"
    )
    conversation_history.append({"role": "user", "content": example_initial_user_prompt})
    conversation_history.append({"role": "assistant", "content": "M0 = A[0,0] * B[0,0]"})

    for step_num in range(max_llm_steps):
        if state.is_c_complete():
            print("  Algorithm complete: Output C is fully populated.")
            break

        prompt_matrix_a_repr = str(state.matrix_a.tolist())
        prompt_matrix_b_repr = str(state.matrix_b.tolist())
        output_c_repr_lines = str(state.output_c).split('\n') # Ensure C is also consistently formatted
        output_c_repr_for_prompt = "\n".join(output_c_repr_lines)


        current_state_summary = (
            f"Current algorithm state:\nMatrices:\nA = {prompt_matrix_a_repr}\nB = {prompt_matrix_b_repr}\n"
            "Executed Steps:\n" + ("\n".join(state.dsl_steps_executed) if state.dsl_steps_executed else "None yet.") + "\n"
            f"Intermediate Variables (M values): {state.intermediate_vars}\n"
            f"Output Matrix C (NaN means not yet computed):\n{output_c_repr_for_prompt}\n"
            "Provide the NEXT SINGLE DSL step according to the rules:"
        )

        user_message = {"role": "user", "content": current_state_summary}
        # Use a slice of history if it gets too long, though Qwen0.5B context is 4096 tokens.
        # For now, using full history.
        # max_history_turns = 5 # Keep last 5 assistant/user pairs + system + initial example
        # if len(conversation_history) > (2 * max_history_turns + 3): # 3 for system + 2 for initial example
        #     prompt_messages_for_llm = [conversation_history[0]] + \
        #                               conversation_history[1:3] + \
        #                               conversation_history[-(2*max_history_turns):] + \
        #                               [user_message]
        # else:
        prompt_messages_for_llm = conversation_history + [user_message]


        llm_dsl_line = get_llm_step_suggestion(prompt_messages_for_llm)

        if llm_dsl_line == "Error: Model not loaded.": # Check specific error string
            state.errors.append(llm_dsl_line)
            print(f"  Stopping due to model loading error string from LLM.")
            break
        if llm_dsl_line == "Error: Cannot determine next step.":
            state.errors.append("LLM indicated it cannot determine next step.")
            print(f"  LLM Output (Step {step_num+1}): '{llm_dsl_line}' -> LLM cannot determine step. Stopping this test case.")
            break

        # Add current turn to history AFTER getting suggestion
        conversation_history.append(user_message)
        conversation_history.append({"role": "assistant", "content": llm_dsl_line}) # Store cleaned line

        if llm_dsl_line.startswith("FORMAT_ERROR:"):
            state.errors.append(f"LLM output format error (cleaned): '{llm_dsl_line}'")
            print(f"  LLM Output (Step {step_num+1}) (cleaned to format error): '{llm_dsl_line}') -> Parse Error. Stopping this test case.")
            break # Stop on format error

        parsed = parse_dsl_step(llm_dsl_line) # Should parse if not FORMAT_ERROR

        if parsed: # If it's not a format error, it should have been parsed by clean_llm_output's check
            if not state.execute_step(parsed, llm_dsl_line):
                # Error already logged in state.execute_step
                print(f"  LLM Output (Step {step_num+1}): '{llm_dsl_line}' -> Parsed, but execution failed. Error: {state.errors[-1] if state.errors else 'Unknown exec error'}. Stopping this test case.")
                # break # Decide if to stop on execution error
            else:
                print(f"  LLM Output (Step {step_num+1}): '{llm_dsl_line}' -> Executed successfully.")
        else:
            # This case should ideally be caught by "FORMAT_ERROR" from clean_llm_output
            state.errors.append(f"LLM output (after cleaning: '{llm_dsl_line}') still could not be parsed by primary parser.")
            print(f"  LLM Output (Step {step_num+1}) (cleaned: '{llm_dsl_line}') -> Internal Parse Error. Stopping this test case.")
            break

    return state

# --- Main Test Loop ---
if model is None:
    print("\nSkipping main test loop as model failed to load.")
else:
    print("\n--- Starting 2x2 Matrix Multiplication Test (Revised v3) ---")
    num_tests = 10 # You can reduce this for faster testing e.g., to 3-5
    correct_algorithms = 0

    for i in range(num_tests):
        print(f"\n--- Test Case {i+1}/{num_tests} ---")
        mat_a_np = np.random.randint(-5, 6, size=(2,2))
        mat_b_np = np.random.randint(-5, 6, size=(2,2))

        print(f"Matrix A:\n{mat_a_np}")
        print(f"Matrix B:\n{mat_b_np}")
        expected_c = np.matmul(mat_a_np, mat_b_np)
        print(f"Expected C:\n{expected_c}")

        final_state = generate_matrix_mult_algorithm(mat_a_np, mat_b_np) # Pass numpy arrays

        print("\nGenerated DSL Algorithm:")
        if final_state.dsl_steps_executed:
            for step_line in final_state.dsl_steps_executed:
                print(step_line)
        else:
            print("No valid DSL steps were executed.")

        if final_state.errors:
            print("\nErrors encountered during generation/execution:")
            # Print unique errors to avoid too much repetition if LLM gets stuck
            unique_errors = []
            for err in final_state.errors:
                if err not in unique_errors:
                    unique_errors.append(err)
            for err in unique_errors:
                print(f"- {err}")


        print(f"\nFinal Computed C (by LLM algorithm):\n{final_state.output_c}")

        if final_state.is_c_complete():
            if final_state.verify_c():
                print("Result: CORRECT")
                correct_algorithms += 1
            else:
                print("Result: INCORRECT (C is complete but values are wrong)")
        else:
            if final_state.dsl_steps_executed: # It tried but didn't finish
                 print("Result: INCOMPLETE (C was not fully computed but some steps were made)")
            else: # No steps made, likely immediate errors
                 print("Result: FAILED TO START (No valid DSL steps executed)")


    print(f"\n--- Test Summary ---")
    print(f"Total correct algorithms: {correct_algorithms}/{num_tests} ({correct_algorithms/num_tests*100:.2f}%)")

Successfully loaded model and tokenizer: Qwen/Qwen1.5-0.5B-Chat
Model is on GPU: cuda:0

--- Starting 2x2 Matrix Multiplication Test (Revised v3) ---

--- Test Case 1/10 ---
Matrix A:
[[-4  5]
 [ 3 -4]]
Matrix B:
[[ 2  2]
 [-4  1]]
Expected C:
[[-28  -3]
 [ 22   2]]
  LLM Output (Step 1) (cleaned to format error): 'FORMAT_ERROR: M2 = B[-4][2] + B [-2][1] - M[2') -> Parse Error. Stopping this test case.

Generated DSL Algorithm:
No valid DSL steps were executed.

Errors encountered during generation/execution:
- LLM output format error (cleaned): 'FORMAT_ERROR: M2 = B[-4][2] + B [-2][1] - M[2'

Final Computed C (by LLM algorithm):
[[nan nan]
 [nan nan]]
Result: FAILED TO START (No valid DSL steps executed)

--- Test Case 2/10 ---
Matrix A:
[[ 1 -2]
 [-3 -1]]
Matrix B:
[[-2 -4]
 [-1 -1]]
Expected C:
[[ 0 -2]
 [ 7 13]]
  LLM Output (Step 1) (cleaned to format error): 'FORMAT_ERROR: Next一步 is to multiply the matrices by the appropriate scalar value.') -> Parse Error. Stopping this test cas

Generate STF

In [ ]:
import random
import json

# --- System Message and DSL Definition (for our reference and inclusion in prompts) ---
SYSTEM_MESSAGE = """ASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B).
NO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE.
If stuck, output ONLY: Error: Cannot determine next step."""

DSL_FORMAT_RULES = """DSL FORMAT:
M<id> = <op1> + <op2>
M<id> = <op1> - <op2>
M<id> = <op1> * <op2>
M<id> = <operand>
C[r,c] = <operand>
(Operands: A[r,c], B[r,c], or M<id>)"""

# --- Dataset Generation ---

def format_matrix_for_prompt(matrix_name, matrix_values):
    return f"{matrix_name} = {matrix_values}"

def format_c_matrix_for_prompt(c_matrix_computed_values):
    c_display = [["nan" for _ in range(2)] for _ in range(2)]
    for key, val_expr_tuple in c_matrix_computed_values.items(): # val_expr_tuple is (value, expression_str)
        # key is like "C[0,0]"
        r, c = int(key[2]), int(key[4])
        # For display, we just show if it's computed, not the value itself in the prompt C matrix
        if val_expr_tuple is not None: # if it has an expression, it's "computed"
             c_display[r][c] = "computed" # Or we could show the M-var, e.g., C[0,0] = M2
    return "Output Matrix C (nan means not yet computed, computed means a DSL line has assigned to it):\n" + \
           f"[[{c_display[0][0]}, {c_display[0][1]}],\n" + \
           f" [{c_display[1][0]}, {c_display[1][1]}]]"

def generate_prompt_text(system_msg, matrices, executed_steps, m_vars, c_matrix_computed_values):
    prompt = f"{system_msg}\n\n"
    prompt += "Current algorithm state:\n"
    prompt += "Matrices:\n"
    prompt += format_matrix_for_prompt("A", matrices['A']) + "\n"
    prompt += format_matrix_for_prompt("B", matrices['B']) + "\n"
    prompt += "Executed Steps:\n"
    if not executed_steps:
        prompt += "None yet.\n"
    else:
        for step in executed_steps:
            prompt += step + "\n"
    prompt += "Intermediate Variables (M values):\n"
    if not m_vars:
        prompt += "{}\n"
    else:
        m_vars_str = ", ".join([f"{k}: {v_tuple[1]}" for k, v_tuple in m_vars.items()]) # v_tuple is (value, expression_str)
        prompt += "{" + m_vars_str + "}\n"
    prompt += format_c_matrix_for_prompt(c_matrix_computed_values) + "\n"
    prompt += "Provide the NEXT SINGLE DSL step according to the rules:"
    return prompt

def generate_standard_matmul_steps_for_sft(num_example_sets=3, start_m_index=0):
    sft_examples = []
    current_m_id = start_m_index

    # Define a few A and B matrices to ensure variety if we run multiple sets
    matrix_variations = [
        ({'A': [[2, 3], [1, 4]], 'B': [[5, 1], [2, 6]]}, "Set1"),
        ({'A': [[-1, 0], [2, 1]], 'B': [[4, 2], [-3, 5]]}, "Set2"),
        ({'A': [[1, 1], [1, 1]], 'B': [[2, 2], [2, 2]]}, "Set3"),
        ({'A': [[7, -2], [0, 9]], 'B': [[1, 0], [0, 1]]}, "Set4") # Identity B
    ]

    for i in range(num_example_sets):
        matrices = matrix_variations[i % len(matrix_variations)][0]
        # These are symbolic, actual values don't matter for format training, but help in state tracking
        A_vals = matrices['A']
        B_vals = matrices['B']

        executed_steps_for_set = []
        m_vars_for_set = {} # Stores { "M0": (value, "A[0,0]*B[0,0]"), ... }
        # Stores { "C[0,0]": (value, "M2"), ... }
        c_matrix_computed_values_for_set = {f"C[{r},{c}]": None for r in range(2) for c in range(2)}

        # C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0]
        steps_for_c00 = [
            f"M{current_m_id} = A[0,0] * B[0,0]",
            f"M{current_m_id+1} = A[0,1] * B[1,0]",
            f"M{current_m_id+2} = M{current_m_id} + M{current_m_id+1}",
            f"C[0,0] = M{current_m_id+2}"
        ]
        m_expr_c00 = [
            ("A[0,0] * B[0,0]", (A_vals[0][0] * B_vals[0][0])),
            ("A[0,1] * B[1,0]", (A_vals[0][1] * B_vals[1][0])),
            (f"M{current_m_id} + M{current_m_id+1}", m_vars_for_set.get(f"M{current_m_id}",(0,""))[0] + m_vars_for_set.get(f"M{current_m_id+1}",(0,""))[0]), # Needs update after m_vars populated
            (f"M{current_m_id+2}", m_vars_for_set.get(f"M{current_m_id+2}",(0,""))[0]) # Needs update
        ]

        # C[0,1] = A[0,0]*B[0,1] + A[0,1]*B[1,1]
        steps_for_c01 = [
            f"M{current_m_id+3} = A[0,0] * B[0,1]",
            f"M{current_m_id+4} = A[0,1] * B[1,1]",
            f"M{current_m_id+5} = M{current_m_id+3} + M{current_m_id+4}",
            f"C[0,1] = M{current_m_id+5}"
        ]
        m_expr_c01 = [
            ("A[0,0] * B[0,1]", (A_vals[0][0] * B_vals[0][1])),
            ("A[0,1] * B[1,1]", (A_vals[0][1] * B_vals[1][1])),
            (f"M{current_m_id+3} + M{current_m_id+4}",0), # placeholder
            (f"M{current_m_id+5}",0) # placeholder
        ]

        # C[1,0] = A[1,0]*B[0,0] + A[1,1]*B[1,0]
        steps_for_c10 = [
            f"M{current_m_id+6} = A[1,0] * B[0,0]",
            f"M{current_m_id+7} = A[1,1] * B[1,0]",
            f"M{current_m_id+8} = M{current_m_id+6} + M{current_m_id+7}",
            f"C[1,0] = M{current_m_id+8}"
        ]
        m_expr_c10 = [
            ("A[1,0] * B[0,0]", (A_vals[1][0] * B_vals[0][0])),
            ("A[1,1] * B[1,0]", (A_vals[1][1] * B_vals[1][0])),
            (f"M{current_m_id+6} + M{current_m_id+7}",0),
            (f"M{current_m_id+8}",0)
        ]

        # C[1,1] = A[1,0]*B[0,1] + A[1,1]*B[1,1]
        steps_for_c11 = [
            f"M{current_m_id+9} = A[1,0] * B[0,1]",
            f"M{current_m_id+10} = A[1,1] * B[1,1]",
            f"M{current_m_id+11} = M{current_m_id+9} + M{current_m_id+10}",
            f"C[1,1] = M{current_m_id+11}"
        ]
        m_expr_c11 = [
            ("A[1,0] * B[0,1]", (A_vals[1][0] * B_vals[0][1])),
            ("A[1,1] * B[1,1]", (A_vals[1][1] * B_vals[1][1])),
            (f"M{current_m_id+9} + M{current_m_id+10}",0),
            (f"M{current_m_id+11}",0)
        ]

        all_steps_for_set = steps_for_c00 + steps_for_c01 + steps_for_c10 + steps_for_c11
        # This logic for m_expr needs to be more dynamic during step processing
        # For simplicity in generation, we'll pre-calculate symbolic expressions for M vars

        # Correctly create (prompt, completion) pairs
        temp_m_id_counter = current_m_id # Used to track M variable names for expressions
        for step_index, target_dsl_step in enumerate(all_steps_for_set):
            # 1. Generate PROMPT based on current state (executed_steps_for_set, m_vars_for_set, c_matrix_computed_values_for_set)
            prompt = generate_prompt_text(SYSTEM_MESSAGE, matrices, executed_steps_for_set, m_vars_for_set, c_matrix_computed_values_for_set)

            # 2. The COMPLETION is the target_dsl_step
            completion = target_dsl_step

            # Add to SFT examples
            # For Qwen, the format is typically ChatML. SFTTrainer can often handle a 'text' field
            # containing the full conversation turn.
            # We will prepare it like this:
            # <|im_start|>system
            # {SYSTEM_MESSAGE}<|im_end|>
            # <|im_start|>user
            # {USER_PROMPT_CONTENT_MINUS_SYSTEM_MESSAGE}<|im_end|>
            # <|im_start|>assistant
            # {DSL_LINE}<|im_end|>
            # The user_prompt_content is what generate_prompt_text creates, MINUS the system message part.
            user_prompt_content = generate_prompt_text("", matrices, executed_steps_for_set, m_vars_for_set, c_matrix_computed_values_for_set) # Pass empty system msg for this part

            # Construct the full text in ChatML format
            chatml_text = f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>\n"
            chatml_text += f"<|im_start|>user\n{user_prompt_content}<|im_end|>\n" # user_prompt_content already has "Current algo state..."
            chatml_text += f"<|im_start|>assistant\n{completion}<|im_end|>"
            sft_examples.append({"text": chatml_text})

            # 3. UPDATE state for the NEXT iteration's prompt
            executed_steps_for_set.append(target_dsl_step)
            parts = target_dsl_step.split(" = ")
            assign_to = parts[0].strip()
            expr_str = parts[1].strip()

            if assign_to.startswith("M"):
                # Simplified: just store the expression string. Actual value computation not needed for format training.
                m_vars_for_set[assign_to] = (None, expr_str) # (value_placeholder, expression_string)
            elif assign_to.startswith("C["):
                c_matrix_computed_values_for_set[assign_to] = (None, expr_str)

        current_m_id += 12 # Increment for the next set of M variables

    # Add a few more examples for other DSL rules if not well covered
    # Example for M<id> = <operand>
    if num_example_sets > 0: # Ensure some state exists
        matrices = matrix_variations[0][0] # Use first matrix set for consistency
        # Assume C[0,0] and M0, M1, M2 are done.
        # Executed: M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], M2=M0+M1, C[0,0]=M2
        _executed_steps = [f"M{start_m_index}=A[0,0]*B[0,0]", f"M{start_m_index+1}=A[0,1]*B[1,0]", f"M{start_m_index+2}=M{start_m_index}+M{start_m_index+1}", f"C[0,0]=M{start_m_index+2}"]
        _m_vars = {
            f"M{start_m_index}": (None, "A[0,0]*B[0,0]"),
            f"M{start_m_index+1}": (None, "A[0,1]*B[1,0]"),
            f"M{start_m_index+2}": (None, f"M{start_m_index}+M{start_m_index+1}")
        }
        _c_vals = {f"C[0,0]": (None, f"M{start_m_index+2}"), "C[0,1]": None, "C[1,0]": None, "C[1,1]": None}

        # 1. M = A[r,c]
        target_dsl = f"M{current_m_id} = A[1,1]"
        user_prompt_content = generate_prompt_text("", matrices, _executed_steps, _m_vars, _c_vals)
        chatml_text = f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>\n<|im_start|>user\n{user_prompt_content}<|im_end|>\n<|im_start|>assistant\n{target_dsl}<|im_end|>"
        sft_examples.append({"text": chatml_text})
        _executed_steps.append(target_dsl)
        _m_vars[f"M{current_m_id}"] = (None, "A[1,1]")
        current_m_id += 1

        # 2. M = M_prev (less common but valid)
        target_dsl = f"M{current_m_id} = M{start_m_index+1}"
        user_prompt_content = generate_prompt_text("", matrices, _executed_steps, _m_vars, _c_vals)
        chatml_text = f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>\n<|im_start|>user\n{user_prompt_content}<|im_end|>\n<|im_start|>assistant\n{target_dsl}<|im_end|>"
        sft_examples.append({"text": chatml_text})
        _executed_steps.append(target_dsl)
        _m_vars[f"M{current_m_id}"] = (None, f"M{start_m_index+1}")
        current_m_id += 1

        # 3. M = op1 - op2
        target_dsl = f"M{current_m_id} = A[0,0] - B[0,0]"
        user_prompt_content = generate_prompt_text("", matrices, _executed_steps, _m_vars, _c_vals)
        chatml_text = f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>\n<|im_start|>user\n{user_prompt_content}<|im_end|>\n<|im_start|>assistant\n{target_dsl}<|im_end|>"
        sft_examples.append({"text": chatml_text})
        # No need to update state further for this example generator

    print(f"Generated {len(sft_examples)} SFT examples.")
    return sft_examples

# Generate the dataset (aim for ~50, 16 per set, so 3 sets = 48 + a few more)
# 3 sets * 16 steps/set = 48 examples. Plus the 3 manual ones = 51.
fine_tuning_data = generate_standard_matmul_steps_for_sft(num_example_sets=3)

# Save to a JSONL file for inspection or use by Hugging Face datasets
output_jsonl_path = "dsl_finetune_data.jsonl"
with open(output_jsonl_path, 'w') as f:
    for item in fine_tuning_data:
        f.write(json.dumps(item) + "\n")

print(f"Fine-tuning data saved to {output_jsonl_path}")
print("First example:")
print(json.dumps(fine_tuning_data[0], indent=2))
print("\nLast example:")
print(json.dumps(fine_tuning_data[-20], indent=2))

Generated 51 SFT examples.
Fine-tuning data saved to dsl_finetune_data.jsonl
First example:
{
  "text": "<|im_start|>system\nASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B).\nNO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE.\nIf stuck, output ONLY: Error: Cannot determine next step.<|im_end|>\n<|im_start|>user\n\n\nCurrent algorithm state:\nMatrices:\nA = [[2, 3], [1, 4]]\nB = [[5, 1], [2, 6]]\nExecuted Steps:\nNone yet.\nIntermediate Variables (M values):\n{}\nOutput Matrix C (nan means not yet computed, computed means a DSL line has assigned to it):\n[[nan, nan],\n [nan, nan]]\nProvide the NEXT SINGLE DSL step according to the rules:<|im_end|>\n<|im_start|>assistant\nM0 = A[0,0] * B[0,0]<|im_end|>"
}

Last example:
{
  "text": "<|im_start|>system\nASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B).\nNO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE.\nIf stuck, output ONLY: Error: Cannot determin

In [ ]:
!pip install trl

  Using cached trl-0.17.0-py3-none-any.whl.metadata (12 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12

In [ ]:
!pip install peft==0.13.2 transformers==4.41.1


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


In [ ]:
# ===== 1. Installs =====
# !pip install -q transformers==4.38.2 datasets peft accelerate bitsandbytes trl

# ===== 2. Imports & Drive Mount =====
import os
import json
import torch
from google.colab import drive
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

# Mount Google Drive for saving
drive.mount('/content/drive')
DRIVE_BASE_PATH = "/content/drive/MyDrive/Qwen_DSL_FineTune"
os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

# ===== 3. Prepare or Upload Your JSONL Dataset =====
if not os.path.exists("dsl_finetune_data.jsonl"):
    def _gen_dummy():
        sys_msg = (
            "ASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix "
            "multiplication. NO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE."
        )
        examples = []
        # Example 1
        user1 = (
            "Current algorithm state:\n"
            "Matrices:\nA = [[2,3],[1,4]] B = [[5,1],[2,6]]\n"
            "Executed Steps: None yet.\n"
            "Output C: [[nan,nan],[nan,nan]]\n"
            "Provide the NEXT SINGLE DSL step:"
        )
        asst1 = "M0 = A[0,0] * B[0,0]"
        text1 = (
            f"<|im_start|>system\n{sys_msg}<|im_end|>\n"
            f"<|im_start|>user\n{user1}<|im_end|>\n"
            f"<|im_start|>assistant\n{asst1}<|im_end|>"
        )
        examples.append({"text": text1})
        # Example 2
        user2 = (
            "Current algorithm state:\n"
            "Matrices:\nA = [[-1,0],[2,1]] B = [[4,2],[-3,5]]\n"
            "Executed Steps:\n"
            "M0 = A[0,0]*B[0,1]\nM1 = A[0,0]*B[0,0]\n"
            "M2 = A[0,1]*B[1,0]\nM3 = A[0,1]*B[1,1]\n"
            "M4 = M1+M2\nM5 = M0+M3\n"
            "Output C: [[nan,nan],[nan,nan]]\n"
            "Provide the NEXT SINGLE DSL step:"
        )
        asst2 = "C[0,1] = M5"
        text2 = (
            f"<|im_start|>system\n{sys_msg}<|im_end|>\n"
            f"<|im_start|>user\n{user2}<|im_end|>\n"
            f"<|im_start|>assistant\n{asst2}<|im_end|>"
        )
        examples.append({"text": text2})
        with open("dsl_finetune_data.jsonl", "w") as f:
            for ex in examples:
                f.write(json.dumps(ex) + "\n")
        print("Dummy dsl_finetune_data.jsonl created.")
    _gen_dummy()

# ===== 4. Load Dataset =====
dataset = load_dataset("json", data_files="dsl_finetune_data.jsonl", split="train")
print(f"Loaded {len(dataset)} examples, e.g.:")
print(dataset[0]["text"])

# ===== 5. Model & Tokenizer Setup =====
MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model = prepare_model_for_kbit_training(base_model)

# ===== 6. PEFT LoRA Config =====
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ===== 7. SFTConfig for Trainer =====
sft_config = SFTConfig(
    output_dir=os.path.join(DRIVE_BASE_PATH, "qwen_dsl_adapter"),
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    bf16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    max_seq_length=512,
    dataset_text_field="text",
    packing=False,
)

# ===== 8. Causal-LM Data Collator =====
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    return_special_tokens_mask=False,
)

# ===== 9. Initialize SFTTrainer =====
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=sft_config,
    processing_class=tokenizer,
    data_collator=data_collator,
)

# ===== 10. Fine-tune =====
print("🚀 Starting fine-tuning...")
trainer.train()
print("🎉 Fine-tuning complete!")

# ===== 11. Save LoRA Adapter & Tokenizer =====
final_adapter_path = os.path.join(DRIVE_BASE_PATH, "qwen_dsl_final_adapter")
trainer.save_model(final_adapter_path)
tokenizer.save_pretrained(final_adapter_path)
print(f"LoRA adapter & tokenizer saved to: {final_adapter_path}")

# ===== 12. (Optional) Quick Test =====
from peft import PeftModel

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
ft_model = PeftModel.from_pretrained(base, final_adapter_path)
ft_model.eval()

system_msg = (
    "ASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication."
)
user_prompt = (
    "Current algorithm state:\n"
    "A=[[2,3],[1,4]] B=[[5,1],[2,6]]\n"
    "Executed Steps: None.\n"
    "Output C: [[nan,nan],[nan,nan]]\n"
    "Provide the NEXT SINGLE DSL step:"
)
messages = [
    {"role": "system", "content": system_msg},
    {"role": "user",   "content": user_prompt},
]
input_text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
inputs = tokenizer([input_text], return_tensors="pt").to(ft_model.device)

with torch.no_grad():
    out = ft_model.generate(
        **inputs,
        max_new_tokens=25,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

generated = tokenizer.decode(
    out[0][inputs["input_ids"].shape[-1]:],
    skip_special_tokens=True
).strip()
print("Generated DSL step:", generated)


RuntimeError: Failed to import trl.trainer.sft_trainer because of the following error (look up to see its traceback):
cannot import name 'BaseImageProcessor' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [ ]:
!pip install --upgrade "transformers>=4.42.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 133.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [ ]:
# ==============================================
# QWEN DSL Fine-Tuning Full Script
# ==============================================
# Dependencies (install in your Colab / environment):
# !pip install -q transformers==4.38.2 datasets peft accelerate bitsandbytes trl

import os
import json
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer

# ------------------------------------------------
# 1. (Optional) Mount Google Drive
# ------------------------------------------------
# from google.colab import drive
# drive.mount('/content/drive')
# DRIVE_BASE_PATH = "/content/drive/MyDrive/Qwen_DSL_FineTune"
# os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

# ------------------------------------------------
# 2. System Message and Dummy Dataset Generation
# ------------------------------------------------
SYSTEM_MESSAGE = (
    "ASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B)."
    "\nNO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE."
    "\nIf stuck, output ONLY: Error: Cannot determine next step."
)

# Create minimal dummy data if JSONL not present
dataset_path = "dsl_finetune_data.jsonl"
if not os.path.exists(dataset_path):
    def _generate_dummy():
        examples = []
        def make_item(user, assistant):
            chatml = (
                f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>"
                f"\n<|im_start|>user\n{user}<|im_end|>"
                f"\n<|im_start|>assistant\n{assistant}<|im_end|>"
            )
            return {"text": chatml}

        # Example 1
        u1 = (
            "Current algorithm state:\n"
            "Matrices:\n"
            "A = [[2, 3], [1, 4]]\n"
            "B = [[5, 1], [2, 6]]\n"
            "Executed Steps:\nNone yet.\n"
            "Intermediate Variables (M values): {}\n"
            "Output Matrix C (NaN means not yet computed):\n"
            "[[nan, nan],\n [nan, nan]]\n"
            "Provide the NEXT SINGLE DSL step according to the rules:"
        )
        a1 = "M0 = A[0,0] * B[0,0]"
        examples.append(make_item(u1, a1))

        # Example 2
        u2 = (
            "Current algorithm state:\n"
            "Matrices:\n"
            "A = [[-1, 0], [2, 1]]\n"
            "B = [[4, 2], [-3, 5]]\n"
            "Executed Steps:\n"
            "M0 = A[0,0] * B[0,1]\nM1 = A[0,0] * B[0,0]\n"
            "M2 = A[0,1] * B[1,0]\nM3 = A[0,1] * B[1,1]\n"
            "M4 = M1 + M2\nM5 = M0 + M3\n"
            "Intermediate Variables (M values): {M0: A[0,0]*B[0,1], M1: A[0,0]*B[0,0], M2: A[0,1]*B[1,0], M3: A[0,1]*B[1,1], M4: M1+M2, M5: M0+M3}\n"
            "Output Matrix C (NaN means not yet computed):\n"
            "[[nan, nan],\n [nan, nan]]\n"
            "Provide the NEXT SINGLE DSL step according to the rules:"
        )
        a2 = "C[0,1] = M5"
        examples.append(make_item(u2, a2))

        with open(dataset_path, 'w') as f:
            for ex in examples:
                f.write(json.dumps(ex) + "\n")
        print(f"Dummy dataset created at {dataset_path}")

    _generate_dummy()

# ------------------------------------------------
# 3. Load Dataset
# ------------------------------------------------
dataset = load_dataset("json", data_files=dataset_path, split="train")
print(f"Loaded {len(dataset)} examples from {dataset_path}")

# ------------------------------------------------
# 4. Model & Tokenizer Setup
# ------------------------------------------------
model_name = "Qwen/Qwen1.5-0.5B-Chat"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

# ------------------------------------------------
# 5. LoRA Configuration & PEFT Wrapping
# ------------------------------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ------------------------------------------------
# 6. Sanity Check: Ensure target_modules exist
# ------------------------------------------------
print("\nValidating target module names:")
targets = lora_config.target_modules
leafs = {name.split('.')[-1] for name, _ in model.named_modules()}
for t in targets:
    print(f"{t:10s} → {('FOUND' if t in leafs else 'MISSING')}" )

# ------------------------------------------------
# 7. Training Arguments & SFTTrainer
# ------------------------------------------------
output_dir = "./qwen_dsl_finetuned_adapter"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=False,
    bf16=True,
    optim="paged_adamw_8bit",
    report_to="none",

)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    # max_seq_length=training_args.max_seq_length,
    max_seq_length=512,
)

# ------------------------------------------------
# 8. Start Fine-Tuning
# ------------------------------------------------
print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning completed.")

# ------------------------------------------------
# 9. Save LoRA Adapter & Tokenizer
# ------------------------------------------------
adapter_path = os.path.join(output_dir, "final_adapter")
trainer.save_model(adapter_path)
tokenizer.save_pretrained(adapter_path)
print(f"Adapter and tokenizer saved at {adapter_path}")

# ------------------------------------------------
# 10. (Optional) Test the Fine-Tuned Model
# ------------------------------------------------
def test_model(prompt: str):
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user",   "content": prompt},
    ]
    text_in = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text_in], return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=25,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    gen = out[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(gen, skip_special_tokens=True).strip()

# Example tests
prompt1 = (
    "Current algorithm state:\nMatrices:\nA = [[2, 3], [1, 4]]\n"
    "B = [[5, 1], [2, 6]]\nExecuted Steps:\nNone yet.\n"
    "Intermediate Variables: {}\nOutput Matrix C ([[nan,nan],[nan,nan]])\n"
    "Provide the NEXT SINGLE DSL step."
)
print("Test 1 response:", test_model(prompt1))

prompt2 = (
    "Current algorithm state:\nMatrices:\nA = [[-1, 0], [2, 1]]\n"
    "B = [[4, 2], [-3, 5]]\nExecuted Steps:\n"
    "M0 = A[0,0]*B[0,1]\nM1 = A[0,0]*B[0,0]\n"
    "M2 = A[0,1]*B[1,0]\nM3 = A[0,1]*B[1,1]\n"
    "M4 = M1+M2\nM5 = M0+M3\n"
    "Intermediate Variables: {...}\nOutput C ([[nan,nan],[nan,nan]])\n"
    "Provide the NEXT SINGLE DSL step."
)
print("Test 2 response:", test_model(prompt2))


Dummy dataset created at dsl_finetune_data.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Loaded 2 examples from dsl_finetune_data.jsonl
trainable params: 7,569,408 || all params: 471,557,120 || trainable%: 1.6052

Validating target module names:
up_proj    → FOUND
v_proj     → FOUND
k_proj     → FOUND
gate_proj  → FOUND
o_proj     → FOUND
down_proj  → FOUND
q_proj     → FOUND


TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'tokenizer'

In [ ]:
prompt1 = (
    "Current algorithm state:\nMatrices:\nA = [[2, 3], [1, 4]]\n"
    "B = [[5, 1], [2, 6]]\nExecuted Steps:\nNone yet.\n"
    "Intermediate Variables: {}\nOutput Matrix C ([[nan,nan],[nan,nan]])\n"
    "Provide the NEXT SINGLE DSL step."
)
print("Test 1 response:", test_model(prompt1))


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Test 1 response: Step 1: Perform matrix multiplication C = A * B.
Step 2: Calculate the result of the product C.


In [ ]:
import random
import json

# --- System Message and DSL Definition (for our reference and inclusion in prompts) ---
SYSTEM_MESSAGE = """ASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B).
NO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE.
If stuck, output ONLY: Error: Cannot determine next step."""

DSL_FORMAT_RULES = """DSL FORMAT:
M<id> = <op1> + <op2>
M<id> = <op1> - <op2>
M<id> = <op1> * <op2>
M<id> = <operand>
C[r,c] = <operand>
(Operands: A[r,c], B[r,c], or M<id>)"""

# --- Dataset Generation ---

def format_matrix_for_prompt(matrix_name, matrix_values):
    return f"{matrix_name} = {matrix_values}"

def format_c_matrix_for_prompt(c_matrix_computed_values):
    c_display = [["nan" for _ in range(2)] for _ in range(2)]
    for key, val_expr_tuple in c_matrix_computed_values.items(): # val_expr_tuple is (value, expression_str)
        # key is like "C[0,0]"
        r, c = int(key[2]), int(key[4])
        # For display, we just show if it's computed, not the value itself in the prompt C matrix
        if val_expr_tuple is not None: # if it has an expression, it's "computed"
             c_display[r][c] = "computed" # Or we could show the M-var, e.g., C[0,0] = M2
    return "Output Matrix C (nan means not yet computed, computed means a DSL line has assigned to it):\n" + \
           f"[[{c_display[0][0]}, {c_display[0][1]}],\n" + \
           f" [{c_display[1][0]}, {c_display[1][1]}]]"

def generate_prompt_text(system_msg, matrices, executed_steps, m_vars, c_matrix_computed_values):
    prompt = f"{system_msg}\n\n"
    prompt += "Current algorithm state:\n"
    prompt += "Matrices:\n"
    prompt += format_matrix_for_prompt("A", matrices['A']) + "\n"
    prompt += format_matrix_for_prompt("B", matrices['B']) + "\n"
    prompt += "Executed Steps:\n"
    if not executed_steps:
        prompt += "None yet.\n"
    else:
        for step in executed_steps:
            prompt += step + "\n"
    prompt += "Intermediate Variables (M values):\n"
    if not m_vars:
        prompt += "{}\n"
    else:
        m_vars_str = ", ".join([f"{k}: {v_tuple[1]}" for k, v_tuple in m_vars.items()]) # v_tuple is (value, expression_str)
        prompt += "{" + m_vars_str + "}\n"
    prompt += format_c_matrix_for_prompt(c_matrix_computed_values) + "\n"
    prompt += "Provide the NEXT SINGLE DSL step according to the rules:"
    return prompt

def generate_standard_matmul_steps_for_sft(num_example_sets=3, start_m_index=0):
    sft_examples = []
    current_m_id = start_m_index

    # Define a few A and B matrices to ensure variety if we run multiple sets
    matrix_variations = [
        ({'A': [[2, 3], [1, 4]], 'B': [[5, 1], [2, 6]]}, "Set1"),
        ({'A': [[-1, 0], [2, 1]], 'B': [[4, 2], [-3, 5]]}, "Set2"),
        ({'A': [[1, 1], [1, 1]], 'B': [[2, 2], [2, 2]]}, "Set3"),
        ({'A': [[7, -2], [0, 9]], 'B': [[1, 0], [0, 1]]}, "Set4") # Identity B
    ]

    for i in range(num_example_sets):
        matrices = matrix_variations[i % len(matrix_variations)][0]
        # These are symbolic, actual values don't matter for format training, but help in state tracking
        A_vals = matrices['A']
        B_vals = matrices['B']

        executed_steps_for_set = []
        m_vars_for_set = {} # Stores { "M0": (value, "A[0,0]*B[0,0]"), ... }
        # Stores { "C[0,0]": (value, "M2"), ... }
        c_matrix_computed_values_for_set = {f"C[{r},{c}]": None for r in range(2) for c in range(2)}

        # C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0]
        steps_for_c00 = [
            f"M{current_m_id} = A[0,0] * B[0,0]",
            f"M{current_m_id+1} = A[0,1] * B[1,0]",
            f"M{current_m_id+2} = M{current_m_id} + M{current_m_id+1}",
            f"C[0,0] = M{current_m_id+2}"
        ]
        m_expr_c00 = [
            ("A[0,0] * B[0,0]", (A_vals[0][0] * B_vals[0][0])),
            ("A[0,1] * B[1,0]", (A_vals[0][1] * B_vals[1][0])),
            (f"M{current_m_id} + M{current_m_id+1}", m_vars_for_set.get(f"M{current_m_id}",(0,""))[0] + m_vars_for_set.get(f"M{current_m_id+1}",(0,""))[0]), # Needs update after m_vars populated
            (f"M{current_m_id+2}", m_vars_for_set.get(f"M{current_m_id+2}",(0,""))[0]) # Needs update
        ]

        # C[0,1] = A[0,0]*B[0,1] + A[0,1]*B[1,1]
        steps_for_c01 = [
            f"M{current_m_id+3} = A[0,0] * B[0,1]",
            f"M{current_m_id+4} = A[0,1] * B[1,1]",
            f"M{current_m_id+5} = M{current_m_id+3} + M{current_m_id+4}",
            f"C[0,1] = M{current_m_id+5}"
        ]
        m_expr_c01 = [
            ("A[0,0] * B[0,1]", (A_vals[0][0] * B_vals[0][1])),
            ("A[0,1] * B[1,1]", (A_vals[0][1] * B_vals[1][1])),
            (f"M{current_m_id+3} + M{current_m_id+4}",0), # placeholder
            (f"M{current_m_id+5}",0) # placeholder
        ]

        # C[1,0] = A[1,0]*B[0,0] + A[1,1]*B[1,0]
        steps_for_c10 = [
            f"M{current_m_id+6} = A[1,0] * B[0,0]",
            f"M{current_m_id+7} = A[1,1] * B[1,0]",
            f"M{current_m_id+8} = M{current_m_id+6} + M{current_m_id+7}",
            f"C[1,0] = M{current_m_id+8}"
        ]
        m_expr_c10 = [
            ("A[1,0] * B[0,0]", (A_vals[1][0] * B_vals[0][0])),
            ("A[1,1] * B[1,0]", (A_vals[1][1] * B_vals[1][0])),
            (f"M{current_m_id+6} + M{current_m_id+7}",0),
            (f"M{current_m_id+8}",0)
        ]

        # C[1,1] = A[1,0]*B[0,1] + A[1,1]*B[1,1]
        steps_for_c11 = [
            f"M{current_m_id+9} = A[1,0] * B[0,1]",
            f"M{current_m_id+10} = A[1,1] * B[1,1]",
            f"M{current_m_id+11} = M{current_m_id+9} + M{current_m_id+10}",
            f"C[1,1] = M{current_m_id+11}"
        ]
        m_expr_c11 = [
            ("A[1,0] * B[0,1]", (A_vals[1][0] * B_vals[0][1])),
            ("A[1,1] * B[1,1]", (A_vals[1][1] * B_vals[1][1])),
            (f"M{current_m_id+9} + M{current_m_id+10}",0),
            (f"M{current_m_id+11}",0)
        ]

        all_steps_for_set = steps_for_c00 + steps_for_c01 + steps_for_c10 + steps_for_c11
        # This logic for m_expr needs to be more dynamic during step processing
        # For simplicity in generation, we'll pre-calculate symbolic expressions for M vars

        # Correctly create (prompt, completion) pairs
        temp_m_id_counter = current_m_id # Used to track M variable names for expressions
        for step_index, target_dsl_step in enumerate(all_steps_for_set):
            # 1. Generate PROMPT based on current state (executed_steps_for_set, m_vars_for_set, c_matrix_computed_values_for_set)
            prompt = generate_prompt_text(SYSTEM_MESSAGE, matrices, executed_steps_for_set, m_vars_for_set, c_matrix_computed_values_for_set)

            # 2. The COMPLETION is the target_dsl_step
            completion = target_dsl_step

            # Add to SFT examples
            # For Qwen, the format is typically ChatML. SFTTrainer can often handle a 'text' field
            # containing the full conversation turn.
            # We will prepare it like this:
            # <|im_start|>system
            # {SYSTEM_MESSAGE}<|im_end|>
            # <|im_start|>user
            # {USER_PROMPT_CONTENT_MINUS_SYSTEM_MESSAGE}<|im_end|>
            # <|im_start|>assistant
            # {DSL_LINE}<|im_end|>
            # The user_prompt_content is what generate_prompt_text creates, MINUS the system message part.
            user_prompt_content = generate_prompt_text("", matrices, executed_steps_for_set, m_vars_for_set, c_matrix_computed_values_for_set) # Pass empty system msg for this part

            # Construct the full text in ChatML format
            chatml_text = f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>\n"
            chatml_text += f"<|im_start|>user\n{user_prompt_content}<|im_end|>\n" # user_prompt_content already has "Current algo state..."
            chatml_text += f"<|im_start|>assistant\n{completion}<|im_end|>"
            sft_examples.append({"text": chatml_text})

            # 3. UPDATE state for the NEXT iteration's prompt
            executed_steps_for_set.append(target_dsl_step)
            parts = target_dsl_step.split(" = ")
            assign_to = parts[0].strip()
            expr_str = parts[1].strip()

            if assign_to.startswith("M"):
                # Simplified: just store the expression string. Actual value computation not needed for format training.
                m_vars_for_set[assign_to] = (None, expr_str) # (value_placeholder, expression_string)
            elif assign_to.startswith("C["):
                c_matrix_computed_values_for_set[assign_to] = (None, expr_str)

        current_m_id += 12 # Increment for the next set of M variables

    # Add a few more examples for other DSL rules if not well covered
    # Example for M<id> = <operand>
    if num_example_sets > 0: # Ensure some state exists
        matrices = matrix_variations[0][0] # Use first matrix set for consistency
        # Assume C[0,0] and M0, M1, M2 are done.
        # Executed: M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], M2=M0+M1, C[0,0]=M2
        _executed_steps = [f"M{start_m_index}=A[0,0]*B[0,0]", f"M{start_m_index+1}=A[0,1]*B[1,0]", f"M{start_m_index+2}=M{start_m_index}+M{start_m_index+1}", f"C[0,0]=M{start_m_index+2}"]
        _m_vars = {
            f"M{start_m_index}": (None, "A[0,0]*B[0,0]"),
            f"M{start_m_index+1}": (None, "A[0,1]*B[1,0]"),
            f"M{start_m_index+2}": (None, f"M{start_m_index}+M{start_m_index+1}")
        }
        _c_vals = {f"C[0,0]": (None, f"M{start_m_index+2}"), "C[0,1]": None, "C[1,0]": None, "C[1,1]": None}

        # 1. M = A[r,c]
        target_dsl = f"M{current_m_id} = A[1,1]"
        user_prompt_content = generate_prompt_text("", matrices, _executed_steps, _m_vars, _c_vals)
        chatml_text = f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>\n<|im_start|>user\n{user_prompt_content}<|im_end|>\n<|im_start|>assistant\n{target_dsl}<|im_end|>"
        sft_examples.append({"text": chatml_text})
        _executed_steps.append(target_dsl)
        _m_vars[f"M{current_m_id}"] = (None, "A[1,1]")
        current_m_id += 1

        # 2. M = M_prev (less common but valid)
        target_dsl = f"M{current_m_id} = M{start_m_index+1}"
        user_prompt_content = generate_prompt_text("", matrices, _executed_steps, _m_vars, _c_vals)
        chatml_text = f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>\n<|im_start|>user\n{user_prompt_content}<|im_end|>\n<|im_start|>assistant\n{target_dsl}<|im_end|>"
        sft_examples.append({"text": chatml_text})
        _executed_steps.append(target_dsl)
        _m_vars[f"M{current_m_id}"] = (None, f"M{start_m_index+1}")
        current_m_id += 1

        # 3. M = op1 - op2
        target_dsl = f"M{current_m_id} = A[0,0] - B[0,0]"
        user_prompt_content = generate_prompt_text("", matrices, _executed_steps, _m_vars, _c_vals)
        chatml_text = f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>\n<|im_start|>user\n{user_prompt_content}<|im_end|>\n<|im_start|>assistant\n{target_dsl}<|im_end|>"
        sft_examples.append({"text": chatml_text})
        # No need to update state further for this example generator

    print(f"Generated {len(sft_examples)} SFT examples.")
    return sft_examples

# Generate the dataset (aim for ~50, 16 per set, so 3 sets = 48 + a few more)
# 3 sets * 16 steps/set = 48 examples. Plus the 3 manual ones = 51.
fine_tuning_data = generate_standard_matmul_steps_for_sft(num_example_sets=30)

# Save to a JSONL file for inspection or use by Hugging Face datasets
output_jsonl_path = "dsl_finetune_data.jsonl"
with open(output_jsonl_path, 'w') as f:
    for item in fine_tuning_data:
        f.write(json.dumps(item) + "\n")

print(f"Fine-tuning data saved to {output_jsonl_path}")
print("First example:")
print(json.dumps(fine_tuning_data[0], indent=2))
print("\nLast example:")
print(json.dumps(fine_tuning_data[-1], indent=2))

Generated 483 SFT examples.
Fine-tuning data saved to dsl_finetune_data.jsonl
First example:
{
  "text": "<|im_start|>system\nASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B).\nNO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE.\nIf stuck, output ONLY: Error: Cannot determine next step.<|im_end|>\n<|im_start|>user\n\n\nCurrent algorithm state:\nMatrices:\nA = [[2, 3], [1, 4]]\nB = [[5, 1], [2, 6]]\nExecuted Steps:\nNone yet.\nIntermediate Variables (M values):\n{}\nOutput Matrix C (nan means not yet computed, computed means a DSL line has assigned to it):\n[[nan, nan],\n [nan, nan]]\nProvide the NEXT SINGLE DSL step according to the rules:<|im_end|>\n<|im_start|>assistant\nM0 = A[0,0] * B[0,0]<|im_end|>"
}

Last example:
{
  "text": "<|im_start|>system\nASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B).\nNO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE.\nIf stuck, output ONLY: Error: Cannot determi

Test with both correct traning code and data

In [ ]:
# ==============================================
# QWEN DSL Fine-Tuning Full Script
# ==============================================
# Dependencies (install in your Colab / environment):
# !pip install -q transformers datasets peft accelerate bitsandbytes trl

import os
import json
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer

# ------------------------------------------------
# 1. (Optional) Mount Google Drive
# ------------------------------------------------
# from google.colab import drive
# drive.mount('/content/drive')
# DRIVE_BASE_PATH = "/content/drive/MyDrive/Qwen_DSL_FineTune"
# os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

# ------------------------------------------------
# 2. System Message and Dummy Dataset Generation
# ------------------------------------------------
SYSTEM_MESSAGE = (
    "ASSISTANT'S ONLY TASK: Output ONE SINGLE VALID DSL line for 2x2 matrix multiplication (C = A * B)."
    "\nNO PYTHON. NO EXPLANATIONS. ONLY THE DSL LINE."
    "\nIf stuck, output ONLY: Error: Cannot determine next step."
)

# Create minimal dummy data if JSONL not present
dataset_path = "dsl_finetune_data.jsonl"
if not os.path.exists(dataset_path):
    def _generate_dummy():
        examples = []
        def make_item(user, assistant):
            chatml = (
                f"<|im_start|>system\n{SYSTEM_MESSAGE}<|im_end|>"
                f"\n<|im_start|>user\n{user}<|im_end|>"
                f"\n<|im_start|>assistant\n{assistant}<|im_end|>"
            )
            return {"text": chatml}

        # Example 1
        u1 = (
            "Current algorithm state:\n"
            "Matrices:\n"
            "A = [[2, 3], [1, 4]]\n"
            "B = [[5, 1], [2, 6]]\n"
            "Executed Steps:\nNone yet.\n"
            "Intermediate Variables (M values): {}\n"
            "Output Matrix C (NaN means not yet computed):\n"
            "[[nan, nan],\n [nan, nan]]\n"
            "Provide the NEXT SINGLE DSL step according to the rules:"
        )
        a1 = "M0 = A[0,0] * B[0,0]"
        examples.append(make_item(u1, a1))

        # Example 2
        u2 = (
            "Current algorithm state:\n"
            "Matrices:\n"
            "A = [[-1, 0], [2, 1]]\n"
            "B = [[4, 2], [-3, 5]]\n"
            "Executed Steps:\n"
            "M0 = A[0,0] * B[0,1]\nM1 = A[0,0] * B[0,0]\n"
            "M2 = A[0,1] * B[1,0]\nM3 = A[0,1] * B[1,1]\n"
            "M4 = M1 + M2\nM5 = M0 + M3\n"
            "Intermediate Variables (M values): {M0: A[0,0]*B[0,1], M1: A[0,0]*B[0,0], M2: A[0,1]*B[1,0], M3: A[0,1]*B[1,1], M4: M1+M2, M5: M0+M3}\n"
            "Output Matrix C (NaN means not yet computed):\n"
            "[[nan, nan],\n [nan, nan]]\n"
            "Provide the NEXT SINGLE DSL step according to the rules:"
        )
        a2 = "C[0,1] = M5"
        examples.append(make_item(u2, a2))

        with open(dataset_path, 'w') as f:
            for ex in examples:
                f.write(json.dumps(ex) + "\n")
        print(f"Dummy dataset created at {dataset_path}")

    _generate_dummy()

# ------------------------------------------------
# 3. Load Dataset
# ------------------------------------------------
dataset = load_dataset("json", data_files=dataset_path, split="train")
print(f"Loaded {len(dataset)} examples from {dataset_path}")

# ------------------------------------------------
# 4. Model & Tokenizer Setup
# ------------------------------------------------
model_name = "Qwen/Qwen1.5-0.5B-Chat"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

# ------------------------------------------------
# 5. LoRA Configuration & PEFT Wrapping
# ------------------------------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ------------------------------------------------
# 6. Sanity Check: Ensure target_modules exist
# ------------------------------------------------
print("\nValidating target module names:")
targets = lora_config.target_modules
leafs = {name.split('.')[-1] for name, _ in model.named_modules()}
for t in targets:
    print(f"{t:10s} → {('FOUND' if t in leafs else 'MISSING')}")

# ------------------------------------------------
# 7. Training Arguments & SFTTrainer
# ------------------------------------------------
output_dir = "./qwen_dsl_finetuned_adapter"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=False,
    bf16=True,
    optim="paged_adamw_8bit",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
)

# ------------------------------------------------
# 8. Start Fine-Tuning
# ------------------------------------------------
print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning completed.")

# ------------------------------------------------
# 9. Save LoRA Adapter & Tokenizer
# ------------------------------------------------
adapter_path = os.path.join(output_dir, "final_adapter")
trainer.save_model(adapter_path)
tokenizer.save_pretrained(adapter_path)
print(f"Adapter and tokenizer saved at {adapter_path}")

# ------------------------------------------------
# 10. (Optional) Test the Fine-Tuned Model
# ------------------------------------------------
def test_model(prompt: str):
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user",   "content": prompt},
    ]
    text_in = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer([text_in], return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=25,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    gen = out[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(gen, skip_special_tokens=True).strip()

# Example tests
prompt1 = (
    "Current algorithm state:\nMatrices:\nA = [[2, 3], [1, 4]]\n"
    "B = [[5, 1], [2, 6]]\nExecuted Steps:\nNone yet.\n"
    "Intermediate Variables: {}\nOutput Matrix C ([[nan,nan],[nan,nan]])\n"
    "Provide the NEXT SINGLE DSL step."
)
print("Test 1 response:", test_model(prompt1))

prompt2 = (
    "Current algorithm state:\nMatrices:\nA = [[-1, 0], [2, 1]]\n"
    "B = [[4, 2], [-3, 5]]\nExecuted Steps:\n"
    "M0 = A[0,0]*B[0,1]\nM1 = A[0,0]*B[0,0]\n"
    "M2 = A[0,1]*B[1,0]\nM3 = A[0,1]*B[1,1]\n"
    "M4 = M1+M2\nM5 = M0+M3\n"
    "Intermediate Variables: {...}\nOutput C ([[nan,nan],[nan,nan]])\n"
    "Provide the NEXT SINGLE DSL step."
)
print("Test 2 response:", test_model(prompt2))


Generating train split: 0 examples [00:00, ? examples/s]

Loaded 483 examples from dsl_finetune_data.jsonl
trainable params: 7,569,408 || all params: 471,557,120 || trainable%: 1.6052

Validating target module names:
o_proj     → FOUND
k_proj     → FOUND
down_proj  → FOUND
q_proj     → FOUND
up_proj    → FOUND
gate_proj  → FOUND
v_proj     → FOUND


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/483 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


...

Fine-tuning completed.


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Adapter and tokenizer saved at ./qwen_dsl_finetuned_adapter/final_adapter


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Test 1 response: M96 = A[0,0] * B[0,0]
Test 2 response: C[0,0] = M5


In [ ]:
import copy

def get_dsl_test_suite():
    """Generates 10 test cases for DSL-based 2x2 matrix multiplication."""

    ops_definition = [
        {'dsl': "M0 = A[0,0]*B[0,0]", 'var': 'M0', 'calc': lambda A,B,V: A[0][0]*B[0][0]},
        {'dsl': "M1 = A[0,1]*B[1,0]", 'var': 'M1', 'calc': lambda A,B,V: A[0][1]*B[1][0]},
        {'dsl': "S0 = M0+M1",         'var': 'S0', 'calc': lambda A,B,V: V['M0']+V['M1']},
        {'dsl': "C[0,0] = S0",        'var': None, 'calc': lambda A,B,V: V['S0'], 'c_idx': (0,0)},
        {'dsl': "M2 = A[0,0]*B[0,1]", 'var': 'M2', 'calc': lambda A,B,V: A[0][0]*B[0][1]},
        {'dsl': "M3 = A[0,1]*B[1,1]", 'var': 'M3', 'calc': lambda A,B,V: A[0][1]*B[1][1]},
        {'dsl': "S1 = M2+M3",         'var': 'S1', 'calc': lambda A,B,V: V['M2']+V['M3']},
        {'dsl': "C[0,1] = S1",        'var': None, 'calc': lambda A,B,V: V['S1'], 'c_idx': (0,1)},
        {'dsl': "M4 = A[1,0]*B[0,0]", 'var': 'M4', 'calc': lambda A,B,V: A[1][0]*B[0][0]},
        {'dsl': "M5 = A[1,1]*B[1,0]", 'var': 'M5', 'calc': lambda A,B,V: A[1][1]*B[1][0]},
        {'dsl': "S2 = M4+M5",         'var': 'S2', 'calc': lambda A,B,V: V['M4']+V['M5']},
        {'dsl': "C[1,0] = S2",        'var': None, 'calc': lambda A,B,V: V['S2'], 'c_idx': (1,0)},
        {'dsl': "M6 = A[1,0]*B[0,1]", 'var': 'M6', 'calc': lambda A,B,V: A[1][0]*B[0][1]},
        {'dsl': "M7 = A[1,1]*B[1,1]", 'var': 'M7', 'calc': lambda A,B,V: A[1][1]*B[1][1]},
        {'dsl': "S3 = M6+M7",         'var': 'S3', 'calc': lambda A,B,V: V['M6']+V['M7']},
        {'dsl': "C[1,1] = S3",        'var': None, 'calc': lambda A,B,V: V['S3'], 'c_idx': (1,1)},
    ]

    matrix_sets = [
        {"A": [[2,3],[1,4]], "B": [[5,1],[2,6]]},
        {"A": [[-1,0],[2,1]], "B": [[4,2],[-3,5]]},
        {"A": [[1,0],[0,1]], "B": [[10,20],[30,40]]},
        {"A": [[1,1],[1,1]], "B": [[1,2],[3,1]]},
        {"A": [[0,1],[1,0]], "B": [[5,8],[2,7]]}
    ]

    test_points_config = [
        (0, 0), (0, 1), (0, 2), (1, 3), (1, 4),
        (2, 6), (2, 8), (3, 11), (3, 12), (4, 14)
    ]

    test_suite = []

    for i, (matrix_idx, num_executed) in enumerate(test_points_config):
        current_A_val = matrix_sets[matrix_idx]["A"]
        current_B_val = matrix_sets[matrix_idx]["B"]

        matrix_a_str = str(current_A_val).replace(" ", "")
        matrix_b_str = str(current_B_val).replace(" ", "")

        executed_dsl_list_for_prompt = []
        intermediate_vars_dict_for_prompt = {}
        current_C_matrix_val = [['nan' for _ in range(2)] for _ in range(2)]

        for step_idx in range(num_executed):
            op = ops_definition[step_idx]
            executed_dsl_list_for_prompt.append(op['dsl'])
            val = op['calc'](current_A_val, current_B_val, intermediate_vars_dict_for_prompt)
            if op['var']:
                intermediate_vars_dict_for_prompt[op['var']] = val
            elif 'c_idx' in op:
                r, c = op['c_idx']
                current_C_matrix_val[r][c] = str(val)

        executed_steps_str_for_prompt = "\n".join(executed_dsl_list_for_prompt) if executed_dsl_list_for_prompt else "None yet."
        intermediate_vars_str_for_prompt = str(intermediate_vars_dict_for_prompt)
        c_matrix_str_for_prompt = f"([[{current_C_matrix_val[0][0]},{current_C_matrix_val[0][1]}],[{current_C_matrix_val[1][0]},{current_C_matrix_val[1][1]}]])"

        prompt_str = (
            f"Current algorithm state:\nMatrices:\nA = {matrix_a_str}\n"
            f"B = {matrix_b_str}\nExecuted Steps:\n{executed_steps_str_for_prompt}\n"
            f"Intermediate Variables: {intermediate_vars_str_for_prompt}\nOutput Matrix C {c_matrix_str_for_prompt}\n"
            "Provide the NEXT SINGLE DSL step."
        )

        expected_dsl_step_str = ops_definition[num_executed]['dsl']

        test_suite.append({
            "id": i + 1,
            "prompt": prompt_str,
            "expected_dsl_step": expected_dsl_step_str,
        })

    return test_suite

# Ensure your 'test_model' function is defined elsewhere in your script
# or imported before this __main__ block is executed.
# For example:
# from your_module import test_model

if __name__ == '__main__':
    dsl_tests = get_dsl_test_suite()

    print("\nRunning DSL Generation Tests (calling your existing 'test_model' function):")
    correct_predictions = 0
    total_tests = len(dsl_tests)

    # Make sure your 'test_model' function is accessible here.
    # If it's not defined globally or imported, this will cause a NameError.
    if 'test_model' not in globals() and 'test_model' not in locals():
        print("\nERROR: 'test_model' function not found.")
        print("Please ensure your 'test_model(prompt_string)' function is defined or imported.")
        print("Skipping test execution.\n")
    else:
        for test_case in dsl_tests:
            print(f"\n----- Test Case {test_case['id']} -----")
            # print(f"Full Prompt:\n{test_case['prompt']}\n") # Uncomment to see the full prompt if needed

            # Call your existing model function with the prompt
            actual_dsl_step = test_model(test_case['prompt']) # This now calls YOUR function

            print(f"Test {test_case['id']} response: {actual_dsl_step}")
            print(f"Expected DSL Step   : {test_case['expected_dsl_step']}")

            if actual_dsl_step == test_case['expected_dsl_step']:
                print("Result: CORRECT ✅")
                correct_predictions += 1
            else:
                print("Result: INCORRECT ❌")
                # You might want to print the full prompt here if it's incorrect for debugging
                # print(f"  Full Prompt for Incorrect Test {test_case['id']}:\n{test_case['prompt']}")

        print("\n----- Test Summary -----")
        print(f"Total tests run: {total_tests}")
        print(f"Correct predictions: {correct_predictions}")
        print(f"Accuracy: {correct_predictions/total_tests:.2%}" if total_tests > 0 else "N/A")
        print("-------------------------\n")

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Running DSL Generation Tests (calling your existing 'test_model' function):

----- Test Case 1 -----


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Test 1 response: M96 = A[0,0] * B[0,0]
Expected DSL Step   : M0 = A[0,0]*B[0,0]
Result: INCORRECT ❌

----- Test Case 2 -----


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Test 7 response: M4 = A[1,0]*B[0,0]
M5 = A[1,1]*B
Expected DSL Step   : M4 = A[1,0]*B[0,0]
Result: INCORRECT ❌

----- Test Case 8 -----


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Test 8 response: M6 = A[1,0]*B[0,1]
Expected DSL Step   : C[1,0] = S2
Result: INCORRECT ❌

----- Test Case 9 -----


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Test 9 response: M6 = A[1,0]*B[0,1]
Expected DSL Step   : M6 = A[1,0]*B[0,1]
Result: CORRECT ✅

----- Test Case 10 -----
Test 10 response: M7 = A[1,1]*B[1,1]
Expected DSL Step   : S3 = M6+M7
Result: INCORRECT ❌

----- Test Summary -----
Total tests run: 10
Correct predictions: 2
Accuracy: 20.00%
-------------------------



## Training the model for normal matrix mul using Finetune

In [ ]:
!pip install --upgrade "transformers==4.42.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

In [ ]:
!pip install -q datasets accelerate bitsandbytes trl
!pip install peft==0.13.2
!pip install --upgrade "transformers>=4.42.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import random
import json

# This COMPREHENSIVE_SYSTEM_MESSAGE_FOR_FULL_SEQUENCE defines all potential DSL rules.
# The fine-tuning data (assistant responses) will teach the specific algorithm and M/S naming conventions.
COMPREHENSIVE_SYSTEM_MESSAGE_FOR_FULL_SEQUENCE = """You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fine-tuning examples (assistant responses) will demonstrate the specific M-variable and S-variable naming convention (M for products, S for sums).
5.  If you cannot determine a valid full DSL sequence based on the rules and input, output ONLY the exact text: Error: Cannot determine full sequence.

DSL FORMAT DEFINITIONS:

1.  **Intermediate Variable Assignment (M-variables):**
    * Purpose: To store intermediate results from arithmetic operations or direct assignments.
    * Naming: `<id>` is an integer (e.g., M0, M1, M2, ...). Use the next available or contextually appropriate ID. (The specific M/S convention for matmul will be learned from examples).
    * Formats:
        * `M<id> = <operand1> + <operand2>`  (Addition)
        * `M<id> = <operand1> - <operand2>`  (Subtraction)
        * `M<id> = <operand1> * <operand2>`  (Multiplication of operands)
        * `M<id> = <operand>`             (Direct assignment)
    * Examples:
        * `M0 = A[0,0] + B[0,0]`
        * `M1 = A[0,1] - M0`
        * `M2 = A[0,0] * B[0,0]`
        * `M3 = M1`
        * `M4 = B[1,0]`

2.  **Output Matrix Cell Assignment (C-matrix):**
    * Purpose: To assign a computed value to an element of the 2x2 output matrix C.
    * Format: `C[<row>,<col>] = <operand>`
    * Constraints: `<row>` and `<col>` must be 0 or 1.
    * Example: `C[0,1] = M5` (Note: for standard matmul, the operand will typically be an S-variable as learned from examples like S0, S1, etc.)

3.  **Operands (`<operand>`, `<operand1>`, `<operand2>`):**
    * Can be an element from input matrix A: `A[<row>,<col>]` (e.g., A[0,0], A[1,0])
    * Can be an element from input matrix B: `B[<row>,<col>]` (e.g., B[0,1], B[1,1])
    * Can be a previously defined intermediate variable: `M<id>` (e.g., M0, M3). (The examples will show S-variables like S0, S1, etc., being used for sums in the matmul context).
    * Constraints: All `<row>` and `<col>` indices for A and B must be 0 or 1. Any `M<id>` (or S-variable, which is a type of intermediate variable) used as an operand must have been defined in a previous DSL step in the sequence.

You will be given the input matrices A (2x2) and B (2x2).
Provide the complete, multi-line DSL sequence to calculate C = A * B.
"""

def get_standard_ops_definition():
    """
    Defines the 16 DSL steps for a standard 2x2 matrix multiplication.
    The 'dsl' field provides the string for each step.
    This sequence implicitly defines the M-for-product and S-for-sum convention.
    """
    ops = [
        # C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0]
        {'dsl': "M0 = A[0,0]*B[0,0]", 'var_out': 'M0', 'symbolic_rhs': "A[0,0]*B[0,0]"},
        {'dsl': "M1 = A[0,1]*B[1,0]", 'var_out': 'M1', 'symbolic_rhs': "A[0,1]*B[1,0]"},
        {'dsl': "S0 = M0+M1",         'var_out': 'S0', 'symbolic_rhs': "M0+M1"},
        {'dsl': "C[0,0] = S0",        'var_out': None, 'c_idx': (0,0), 'symbolic_rhs': "S0"},

        # C[0,1] = A[0,0]*B[0,1] + A[0,1]*B[1,1]
        {'dsl': "M2 = A[0,0]*B[0,1]", 'var_out': 'M2', 'symbolic_rhs': "A[0,0]*B[0,1]"},
        {'dsl': "M3 = A[0,1]*B[1,1]", 'var_out': 'M3', 'symbolic_rhs': "A[0,1]*B[1,1]"},
        {'dsl': "S1 = M2+M3",         'var_out': 'S1', 'symbolic_rhs': "M2+M3"},
        {'dsl': "C[0,1] = S1",        'var_out': None, 'c_idx': (0,1), 'symbolic_rhs': "S1"},

        # C[1,0] = A[1,0]*B[0,0] + A[1,1]*B[1,0]
        {'dsl': "M4 = A[1,0]*B[0,0]", 'var_out': 'M4', 'symbolic_rhs': "A[1,0]*B[0,0]"},
        {'dsl': "M5 = A[1,1]*B[1,0]", 'var_out': 'M5', 'symbolic_rhs': "A[1,1]*B[1,0]"},
        {'dsl': "S2 = M4+M5",         'var_out': 'S2', 'symbolic_rhs': "M4+M5"},
        {'dsl': "C[1,0] = S2",        'var_out': None, 'c_idx': (1,0), 'symbolic_rhs': "S2"},

        # C[1,1] = A[1,0]*B[0,1] + A[1,1]*B[1,1]
        {'dsl': "M6 = A[1,0]*B[0,1]", 'var_out': 'M6', 'symbolic_rhs': "A[1,0]*B[0,1]"},
        {'dsl': "M7 = A[1,1]*B[1,1]", 'var_out': 'M7', 'symbolic_rhs': "A[1,1]*B[1,1]"},
        {'dsl': "S3 = M6+M7",         'var_out': 'S3', 'symbolic_rhs': "M6+M7"},
        {'dsl': "C[1,1] = S3",        'var_out': None, 'c_idx': (1,1), 'symbolic_rhs': "S3"},
    ]
    # Other keys like 'calc_val' are kept for potential other uses but not strictly needed
    # for this specific dataset generation task where only 'dsl' is used for the completion.
    return ops

def generate_random_matrix():
    """Generates a 2x2 matrix with random integer elements between -9 and 9."""
    return [[random.randint(-9, 9) for _ in range(2)] for _ in range(2)]

def generate_sft_dataset_full_sequence(num_full_multiplications=100, system_message=COMPREHENSIVE_SYSTEM_MESSAGE_FOR_FULL_SEQUENCE):
    """
    Generates a fine-tuning dataset for DSL-based 2x2 matrix multiplication.
    Each example is a full multiplication problem (A, B inputs) and the complete DSL sequence as completion.
    Formatted in ChatML.
    """
    ops_definition = get_standard_ops_definition()
    # The assistant's completion is the fixed sequence of DSL steps for standard 2x2 matmul
    full_dsl_completion = "\n".join([op['dsl'] for op in ops_definition])

    sft_data_chatml = [] # Store ChatML formatted strings

    for i in range(num_full_multiplications):
        # Generate new A and B matrices for each full multiplication problem
        if i == 0: current_A_val, current_B_val = [[1,0],[0,1]], generate_random_matrix()
        elif i == 1: current_A_val, current_B_val = generate_random_matrix(), [[1,0],[0,1]]
        elif i == 2: current_A_val, current_B_val = [[0,0],[0,0]], generate_random_matrix()
        elif i == 3: current_A_val, current_B_val = generate_random_matrix(), [[0,0],[0,0]]
        elif i == 4: current_A_val, current_B_val = [[-2, 3],[1, -4]], [[5, -1],[-2, 6]]
        else: current_A_val, current_B_val = generate_random_matrix(), generate_random_matrix()

        matrix_a_str_for_prompt = str(current_A_val).replace(" ", "")
        matrix_b_str_for_prompt = str(current_B_val).replace(" ", "")

        # Construct the "user" part of the prompt
        user_prompt_content = (
            f"Input matrices:\n"
            f"A = {matrix_a_str_for_prompt}\n"
            f"B = {matrix_b_str_for_prompt}\n"
            f"Provide the COMPLETE sequence of DSL steps to calculate C = A * B."
        )

        # Format into ChatML and add to dataset
        chatml_text = f"<|im_start|>system\n{system_message.strip()}<|im_end|>\n"
        chatml_text += f"<|im_start|>user\n{user_prompt_content.strip()}<|im_end|>\n"
        chatml_text += f"<|im_start|>assistant\n{full_dsl_completion}<|im_end|>"
        sft_data_chatml.append({"text": chatml_text})

    return sft_data_chatml

if __name__ == '__main__':
    num_multiplication_examples = 100 # Number of full 2x2 matrix multiplication problems for the dataset

    sft_dataset_chatml = generate_sft_dataset_full_sequence(
        num_multiplication_examples,
        system_message=COMPREHENSIVE_SYSTEM_MESSAGE_FOR_FULL_SEQUENCE
    )

    print(f"Generated {len(sft_dataset_chatml)} ChatML examples for SFT.")
    print(f"Each example provides input matrices A and B, and expects the full {len(get_standard_ops_definition())}-step DSL sequence as the completion.")

    print("\n----- Example SFT Data (First ChatML item) -----")
    if sft_dataset_chatml:
        print(sft_dataset_chatml[0]['text'])
        print("--------------------------------------------------")

    if len(sft_dataset_chatml) > 1:
        print("\n----- Example SFT Data (Second ChatML item, if generated) -----")
        print(sft_dataset_chatml[1]['text'])
        print("--------------------------------------------------")

    output_jsonl_path = "dsl_finetune_data_standard_matmul_full_sequence.jsonl"
    with open(output_jsonl_path, 'w') as f:
        for item in sft_dataset_chatml:
            f.write(json.dumps(item) + "\n")
    print(f"\nFine-tuning data saved to {output_jsonl_path}")

Generated 100 ChatML examples for SFT.
Each example provides input matrices A and B, and expects the full 16-step DSL sequence as the completion.

----- Example SFT Data (First ChatML item) -----
<|im_start|>system
You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fin

In [ ]:
# ==============================================
# QWEN DSL Fine-Tuning Full Script (Corrected)
# ==============================================
# Dependencies (install in your Colab / environment):
# !pip install -q datasets accelerate bitsandbytes trl

import os
import json
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig # TrainingArguments not strictly needed if all args in SFTConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training # PeftModel not used in training script
from trl import SFTConfig, SFTTrainer

os.environ["WANDB_DISABLED"] = "true"

# ------------------------------------------------
# 1. (Optional) Mount Google Drive
# ------------------------------------------------
# from google.colab import drive
# drive.mount('/content/drive')
# DRIVE_BASE_PATH = "/content/drive/MyDrive/Qwen_DSL_FineTune"
# os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

# ------------------------------------------------
# 2. System Message and Dataset Path
# ------------------------------------------------
SYSTEM_MESSAGE = (
   """You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fine-tuning examples (assistant responses) will demonstrate the specific M-variable and S-variable naming convention (M for products, S for sums).
5.  If you cannot determine a valid full DSL sequence based on the rules and input, output ONLY the exact text: Error: Cannot determine full sequence.

DSL FORMAT DEFINITIONS:

1.  **Intermediate Variable Assignment (M-variables):**
    * Purpose: To store intermediate results from arithmetic operations or direct assignments.
    * Naming: `<id>` is an integer (e.g., M0, M1, M2, ...). Use the next available or contextually appropriate ID. (The specific M/S convention for matmul will be learned from examples).
    * Formats:
        * `M<id> = <operand1> + <operand2>`  (Addition)
        * `M<id> = <operand1> - <operand2>`  (Subtraction)
        * `M<id> = <operand1> * <operand2>`  (Multiplication of operands)
        * `M<id> = <operand>`             (Direct assignment)
    * Examples:
        * `M0 = A[0,0] + B[0,0]`
        * `M1 = A[0,1] - M0`
        * `M2 = A[0,0] * B[0,0]`
        * `M3 = M1`
        * `M4 = B[1,0]`

2.  **Output Matrix Cell Assignment (C-matrix):**
    * Purpose: To assign a computed value to an element of the 2x2 output matrix C.
    * Format: `C[<row>,<col>] = <operand>`
    * Constraints: `<row>` and `<col>` must be 0 or 1.
    * Example: `C[0,1] = M5` (Note: for standard matmul, the operand will typically be an S-variable as learned from examples like S0, S1, etc.)

3.  **Operands (`<operand>`, `<operand1>`, `<operand2>`):**
    * Can be an element from input matrix A: `A[<row>,<col>]` (e.g., A[0,0], A[1,0])
    * Can be an element from input matrix B: `B[<row>,<col>]` (e.g., B[0,1], B[1,1])
    * Can be a previously defined intermediate variable: `M<id>` (e.g., M0, M3). (The examples will show S-variables like S0, S1, etc., being used for sums in the matmul context).
    * Constraints: All `<row>` and `<col>` indices for A and B must be 0 or 1. Any `M<id>` (or S-variable, which is a type of intermediate variable) used as an operand must have been defined in a previous DSL step in the sequence.

You will be given the input matrices A (2x2) and B (2x2).
Provide the complete, multi-line DSL sequence to calculate C = A * B."""
)

# Dataset path (ensure this file exists and is correctly formatted)
dataset_path = "/content/dsl_finetune_data_standard_matmul_full_sequence.jsonl"
if not os.path.exists(dataset_path):
  print(f"Dataset file not found: {dataset_path}. Please ensure it exists.")
  # Consider exiting or handling this error more robustly if the file is essential
  # exit()

# ------------------------------------------------
# 3. Load Dataset
# ------------------------------------------------
dataset = load_dataset("json", data_files=dataset_path, split="train")
print(f"Loaded {len(dataset)} examples from {dataset_path}")

# ------------------------------------------------
# 4. Model & Tokenizer Setup
# ------------------------------------------------
model_name = "Qwen/Qwen1.5-0.5B-Chat"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

# ------------------------------------------------
# 5. LoRA Configuration & PEFT Wrapping
# ------------------------------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ------------------------------------------------
# 6. Sanity Check: Ensure target_modules exist
# ------------------------------------------------
print("\nValidating target module names:")
targets = lora_config.target_modules
leafs = {name.split('.')[-1] for name, _ in model.named_modules()}
for t in targets:
    print(f"{t:10s} → {('FOUND' if t in leafs else 'MISSING')}")

# ------------------------------------------------
# 7. Training Configuration and Trainer Initialization
# ------------------------------------------------
output_dir = "./qwen_dsl_finetuned_adapter" # Base directory for all outputs

# SFTConfig inherits from TrainingArguments, so all parameters are set here.
sft_args = SFTConfig(
    # SFT specific parameters
    dataset_text_field="text",         # Field in your dataset that contains the ChatML string
    max_seq_length=1500,                # Max sequence length (corrected from max_length)
    packing=False,                     # Whether to pack multiple short examples if True

    # Standard TrainingArguments parameters integrated here
    output_dir=output_dir,             # Trainer will save checkpoints, logs etc. here
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,     # Effective batch size = 2 * 2 = 4
    learning_rate=2e-4,
    num_train_epochs=5,                # Increased epochs for small dataset (e.g., 25 examples). Adjust as needed.
    logging_steps=10,                  # Log training progress every 10 steps
    save_strategy="epoch",             # Save a checkpoint at the end of each epoch
    bf16=True,                         # Use bfloat16 precision (ensure GPU supports it)
    # fp16=False,                      # If bf16 not supported, set fp16=True
    optim="paged_adamw_8bit",          # Optimizer
    # report_to="none",                # Already handled by os.environ["WANDB_DISABLED"]
    # Add other relevant TrainingArguments as needed:
    # weight_decay=0.01,
    # lr_scheduler_type="cosine",
    # warmup_steps=50,
)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,                # Pass the tokenizer instance directly
    args=sft_args,                      # Pass the comprehensive SFTConfig object
    train_dataset=dataset,
    peft_config=lora_config,
    # dataset_text_field, max_seq_length, packing are sourced from sft_args now
)

# ------------------------------------------------
# 8. Start Fine-Tuning
# ------------------------------------------------
print("Starting fine-tuning with effective arguments...")
trainer.train() # Single, effective train call
print("Fine-tuning completed.")

# ------------------------------------------------
# 9. Save LoRA Adapter & Tokenizer
# ------------------------------------------------
# The adapter will be saved in a subdirectory of output_dir (e.g. output_dir/checkpoint-xxx or output_dir if save_strategy="no")
# To save the final adapter explicitly to a desired path after training:
adapter_path = os.path.join(output_dir, "final_adapter")
os.makedirs(adapter_path, exist_ok=True) # Ensure the directory exists
trainer.save_model(adapter_path)
tokenizer.save_pretrained(adapter_path)
print(f"Final adapter and tokenizer saved at {adapter_path}")

# ------------------------------------------------
# 10. (Optional) Test the Fine-Tuned Model (after training and saving)
# ------------------------------------------------
def test_model(prompt: str, trained_model, trained_tokenizer): # Pass the trained model and tokenizer
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user",   "content": prompt},
    ]
    text_in = trained_tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = trained_tokenizer([text_in], return_tensors="pt").to(trained_model.device)
    with torch.no_grad():
        out = trained_model.generate(
            **inputs,
            max_new_tokens=1500, # Increased from 25 to accommodate full DSL sequence
            do_sample=False,
            pad_token_id=trained_tokenizer.eos_token_id,
            eos_token_id=trained_tokenizer.eos_token_id
        )
    gen_tokens = out[0][inputs["input_ids"].shape[-1]:]
    return trained_tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

# To use the test_model function after training, you would typically load the saved adapter:
# from peft import PeftModel
#
# print("\n--- Testing the fine-tuned model ---")
# # Load the base model again (quantized)
# base_model_for_test = AutoModelForCausalLM.from_pretrained(
# model_name,
# quantization_config=bnb_config,
# device_map="auto",
# trust_remote_code=True
# )
# # Load the PeftModel with the adapter
# loaded_model = PeftModel.from_pretrained(base_model_for_test, adapter_path)
# loaded_model.eval() # Set to evaluation mode
#
# loaded_tokenizer = AutoTokenizer.from_pretrained(adapter_path) # Load tokenizer from adapter path
#
# test_prompt_example = (
#     "Input matrices:\n"
#     "A = [[1,2],[3,4]]\n"
#     "B = [[5,6],[7,8]]\n"
#     "Provide the COMPLETE sequence of DSL steps to calculate C = A * B."
# )
# response = test_model(test_prompt_example, loaded_model, loaded_tokenizer)
# print(f"Test Prompt:\n{test_prompt_example}")
# print(f"\nModel Response:\n{response}")

Generating train split: 0 examples [00:00, ? examples/s]

Loaded 100 examples from /content/dsl_finetune_data_standard_matmul_full_sequence.jsonl


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


trainable params: 7,569,408 || all params: 471,557,120 || trainable%: 1.6052

Validating target module names:
v_proj     → FOUND
up_proj    → FOUND
q_proj     → FOUND
gate_proj  → FOUND
o_proj     → FOUND
down_proj  → FOUND
k_proj     → FOUND


Converting train dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Starting fine-tuning with effective arguments...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.457800
20,0.339600
30,0.047300
40,0.025300
50,0.023700
60,0.022500
70,0.022200
80,0.021900
90,0.021800
100,0.021700


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Fine-tuning completed.
Final adapter and tokenizer saved at ./qwen_dsl_finetuned_adapter/final_adapter


In [ ]:
 import os
import json
import torch
import random
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig
)
from peft import PeftModel

# Ensure WANDB is disabled if you had it enabled previously for training
os.environ["WANDB_DISABLED"] = "true"

# ------------------------------------------------
# Configuration (Adjust these paths if necessary)
# ------------------------------------------------
BASE_MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat"
# This is the directory where trainer.save_model() saved the adapter
ADAPTER_PATH = "./qwen_dsl_finetuned_adapter/final_adapter" # Output from step 9

# ------------------------------------------------
# System Message (MUST be IDENTICAL to the one used for fine-tuning)
# ------------------------------------------------
SYSTEM_MESSAGE = (
   """You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fine-tuning examples (assistant responses) will demonstrate the specific M-variable and S-variable naming convention (M for products, S for sums).
5.  If you cannot determine a valid full DSL sequence based on the rules and input, output ONLY the exact text: Error: Cannot determine full sequence.

DSL FORMAT DEFINITIONS:

1.  **Intermediate Variable Assignment (M-variables):**
    * Purpose: To store intermediate results from arithmetic operations or direct assignments.
    * Naming: `<id>` is an integer (e.g., M0, M1, M2, ...). Use the next available or contextually appropriate ID. (The specific M/S convention for matmul will be learned from examples).
    * Formats:
        * `M<id> = <operand1> + <operand2>`  (Addition)
        * `M<id> = <operand1> - <operand2>`  (Subtraction)
        * `M<id> = <operand1> * <operand2>`  (Multiplication of operands)
        * `M<id> = <operand>`             (Direct assignment)
    * Examples:
        * `M0 = A[0,0] + B[0,0]`
        * `M1 = A[0,1] - M0`
        * `M2 = A[0,0] * B[0,0]`
        * `M3 = M1`
        * `M4 = B[1,0]`

2.  **Output Matrix Cell Assignment (C-matrix):**
    * Purpose: To assign a computed value to an element of the 2x2 output matrix C.
    * Format: `C[<row>,<col>] = <operand>`
    * Constraints: `<row>` and `<col>` must be 0 or 1.
    * Example: `C[0,1] = M5` (Note: for standard matmul, the operand will typically be an S-variable as learned from examples like S0, S1, etc.)

3.  **Operands (`<operand>`, `<operand1>`, `<operand2>`):**
    * Can be an element from input matrix A: `A[<row>,<col>]` (e.g., A[0,0], A[1,0])
    * Can be an element from input matrix B: `B[<row>,<col>]` (e.g., B[0,1], B[1,1])
    * Can be a previously defined intermediate variable: `M<id>` (e.g., M0, M3). (The examples will show S-variables like S0, S1, etc., being used for sums in the matmul context).
    * Constraints: All `<row>` and `<col>` indices for A and B must be 0 or 1. Any `M<id>` (or S-variable, which is a type of intermediate variable) used as an operand must have been defined in a previous DSL step in the sequence.

You will be given the input matrices A (2x2) and B (2x2).
Provide the complete, multi-line DSL sequence to calculate C = A * B."""
)

# ------------------------------------------------
# Helper functions from your data generation script
# ------------------------------------------------
def get_standard_ops_definition():
    """
    Defines the 16 DSL steps for a standard 2x2 matrix multiplication.
    """
    ops = [
        {'dsl': "M0 = A[0,0]*B[0,0]"}, {'dsl': "M1 = A[0,1]*B[1,0]"},
        {'dsl': "S0 = M0+M1"},         {'dsl': "C[0,0] = S0"},
        {'dsl': "M2 = A[0,0]*B[0,1]"}, {'dsl': "M3 = A[0,1]*B[1,1]"},
        {'dsl': "S1 = M2+M3"},         {'dsl': "C[0,1] = S1"},
        {'dsl': "M4 = A[1,0]*B[0,0]"}, {'dsl': "M5 = A[1,1]*B[1,0]"},
        {'dsl': "S2 = M4+M5"},         {'dsl': "C[1,0] = S2"},
        {'dsl': "M6 = A[1,0]*B[0,1]"}, {'dsl': "M7 = A[1,1]*B[1,1]"},
        {'dsl': "S3 = M6+M7"},         {'dsl': "C[1,1] = S3"},
    ]
    return ops

def generate_random_matrix():
    """Generates a 2x2 matrix with random integer elements between -9 and 9."""
    return [[random.randint(-9, 9) for _ in range(2)] for _ in range(2)]

EXPECTED_DSL_SEQUENCE = "\n".join([op['dsl'] for op in get_standard_ops_definition()])

# ------------------------------------------------
# Load Fine-Tuned Model for Inference
# ------------------------------------------------
def load_model_for_inference(base_model_name, adapter_path):
    print(f"Loading base model: {base_model_name}")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, # Or 8bit if you trained with that
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config, # Use same quantization as training for base
        device_map="auto",
        trust_remote_code=True
    )

    print(f"Loading adapter from: {adapter_path}")
    # Load the PEFT model by specifying the adapter path
    # The `is_trainable=False` is good practice for inference.
    model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)

    print("Merging adapter into the base model...")
    # Merge LoRA layers for faster inference.
    # If you run out of memory, you can skip this and use the PeftModel directly,
    # but inference will be slower.
    model = model.merge_and_unload()

    tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token # Or tokenizer.unk_token
    tokenizer.padding_side = "right" # Important for generation

    print("Model and tokenizer loaded successfully.")
    return model, tokenizer

# ------------------------------------------------
# Generate Model Response
# ------------------------------------------------
def get_model_response(model, tokenizer, user_prompt_content):
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": user_prompt_content},
    ]

    # Apply chat template
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True # Crucial for Qwen and other instruction-tuned models
    )

    inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

    # Generation parameters
    # max_new_tokens needs to be sufficient for the 16 lines of DSL.
    # 16 lines * ~20 chars/line = 320. Add some buffer.
    # Using do_sample=False for deterministic output, good for testing exact match.
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2000, # Increased for full sequence
            do_sample=False,    # For deterministic output
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id # Model should stop after generating its response
        )

    # Decode only the generated part
    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    response_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response_text.strip()

# ------------------------------------------------
# Main Testing Logic
# ------------------------------------------------
if __name__ == "__main__":
    if not os.path.exists(ADAPTER_PATH):
        print(f"ERROR: Adapter path not found: {ADAPTER_PATH}")
        print("Please ensure you have trained the model and the adapter is saved correctly.")
        exit(1)

    model, tokenizer = load_model_for_inference(BASE_MODEL_NAME, ADAPTER_PATH)
    model.eval() # Set model to evaluation mode

    num_test_cases = 10
    correct_predictions = 0

    print(f"\n--- Starting {num_test_cases} Test Cases ---")
    print(f"Expected DSL Sequence (all test cases):\n{EXPECTED_DSL_SEQUENCE}\n")

    for i in range(num_test_cases):
        print(f"--- Test Case {i+1}/{num_test_cases} ---")

        # Generate random matrices for the prompt
        matrix_a = generate_random_matrix()
        matrix_b = generate_random_matrix()

        matrix_a_str = str(matrix_a).replace(" ", "")
        matrix_b_str = str(matrix_b).replace(" ", "")

        user_prompt = (
            f"Input matrices:\n"
            f"A = {matrix_a_str}\n"
            f"B = {matrix_b_str}\n"
            f"Provide the COMPLETE sequence of DSL steps to calculate C = A * B."
        )
        print(f"User Prompt (content part):\n{user_prompt}")

        generated_dsl = get_model_response(model, tokenizer, user_prompt)

        print(f"\nModel's Generated DSL:\n{generated_dsl}")

        # Normalize whitespace for comparison (e.g., handle potential trailing newlines)
        normalized_generated = "\n".join([line.strip() for line in generated_dsl.strip().split('\n')])
        normalized_expected = "\n".join([line.strip() for line in EXPECTED_DSL_SEQUENCE.strip().split('\n')])

        if normalized_generated == normalized_expected:
            print("\nResult: CORRECT")
            correct_predictions += 1
        else:
            print("\nResult: INCORRECT")
            # For detailed diff, you could use difflib
            # import difflib
            # diff = difflib.unified_diff(
            #     normalized_expected.splitlines(keepends=True),
            #     normalized_generated.splitlines(keepends=True),
            #     fromfile='expected',
            #     tofile='generated',
            # )
            # print("Diff:")
            # print(''.join(diff))
            if len(normalized_generated.split('\n')) != len(normalized_expected.split('\n')):
                print(f"""Length Mismatch: Expected {len(normalized_expected.split('     '))} lines, Got {len(normalized_generated.split('     '))} lines.""")


        print("-" * 30)

    accuracy = (correct_predictions / num_test_cases) * 100
    print(f"\n--- Test Summary ---")
    print(f"Total Test Cases: {num_test_cases}")
    print(f"Correct Predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")

    # Clean up model from GPU memory if necessary
    del model
    del tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

Loading base model: Qwen/Qwen1.5-0.5B-Chat
Loading adapter from: ./qwen_dsl_finetuned_adapter/final_adapter
Merging adapter into the base model...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model and tokenizer loaded successfully.

--- Starting 10 Test Cases ---
Expected DSL Sequence (all test cases):
M0 = A[0,0]*B[0,0]
M1 = A[0,1]*B[1,0]
S0 = M0+M1
C[0,0] = S0
M2 = A[0,0]*B[0,1]
M3 = A[0,1]*B[1,1]
S1 = M2+M3
C[0,1] = S1
M4 = A[1,0]*B[0,0]
M5 = A[1,1]*B[1,0]
S2 = M4+M5
C[1,0] = S2
M6 = A[1,0]*B[0,1]
M7 = A[1,1]*B[1,1]
S3 = M6+M7
C[1,1] = S3

--- Test Case 1/10 ---
User Prompt (content part):
Input matrices:
A = [[-6,-9],[-1,-2]]
B = [[-2,-5],[-6,8]]
Provide the COMPLETE sequence of DSL steps to calculate C = A * B.


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Model's Generated DSL:
Here's the complete sequence of DSL steps to calculate C = A * B:
```
M0 = A[0,0] + B[0,0]
M1 = A[0,1] - M0
M2 = A[0,0] * B[0,0]
M3 = M1
S0 = M0 + M1
C = M0 * M1
```

Explanation:

  * `M0 = A[0,0] + B[0,0]`: This line calculates the sum of the first two elements of the input matrices A and B.
  * `M1 = A[0,1] - M0`: This line calculates the difference between the second and third elements of the input matrices A and B.
  * `M2 = A[0,0] * B[0,0]:` This line calculates the product of the first and third elements of the input matrices A and B.
  * `M3 = M1`: This line calculates the final result of the calculation, which is the computed value of A * B.

Note that the values returned by these steps are not necessarily the same as the actual values calculated by the matmul operation. However, they represent the computed value of A * B.

Result: INCORRECT
Length Mismatch: Expected 1 lines, Got 1 lines.
------------------------------
--- Test Case 2/10 ---
User Prompt

...


Model's Generated DSL:
Here's the complete sequence of DSL steps to calculate C = A * B:
```
M0 = A[0,0] + B[0,0]
M1 = A[0,1] - M0
M2 = A[0,0] * B[0,0]
M3 = M1
S0 = M0 + M1
C = M0 + M1
```

Explanation:

  * `M0 = A[0,0] + B[0,0]`: This line calculates the sum of the first two elements of the input matrices A and B.
  * `M1 = A[0,1] - M0`: This line calculates the difference between the second and third elements of the input matrices A and B.
  * `M2 = A[0,0] * B[0,0]:` This line calculates the product of the first and third elements of the input matrices A and B.
  * `M3 = M1`: This line calculates the final result of the calculation, which is the computed value of A * B.

Note that the values returned by these steps are not necessarily the same as the actual values calculated by the matmul operation. However, they represent the computed value of A * B.

Result: INCORRECT
Length Mismatch: Expected 1 lines, Got 1 lines.
------------------------------
--- Test Case 8/10 ---
User Prompt

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Model's Generated DSL:
Here's the complete sequence of DSL steps to calculate C = A * B:
```
M0 = A[0,0] + B[0,0]
M1 = A[0,1] - M0
M2 = A[0,0] * B[0,0]
M3 = M1
S0 = M0 + M1
C = M0 * M1
```

Explanation:

  * `M0 = A[0,0] + B[0,0]`: This line calculates the sum of the first two elements of the input matrices A and B.
  * `M1 = A[0,1] - M0`: This line calculates the difference between the second and third elements of the input matrices A and B.
  * `M2 = A[0,0] * B[0,0]:` This line calculates the product of the first and third elements of the input matrices A and B.
  * `M3 = M1`: This line calculates the final result of the calculation, which is the computed value of A * B.

Note that the values returned by these steps are not necessarily the same as the actual values calculated by the matmul operation. However, they represent the computed value of A * B.

Result: INCORRECT
Length Mismatch: Expected 1 lines, Got 1 lines.
------------------------------
--- Test Case 9/10 ---
User Prompt

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Model's Generated DSL:
Here's the complete sequence of DSL steps to calculate C = A * B:
```
M0 = A[0,0] + B[0,0]
M1 = A[0,1] - M0
M2 = A[0,0] * B[0,0]
M3 = M1
S0 = M0 + M1
C = M0 * M1
```

Explanation:

  * `M0 = A[0,0] + B[0,0]`: This line calculates the sum of the first two elements of A and B.
  * `M1 = A[0,1] - M0`: This line calculates the difference between the second and third elements of A and B.
  * `M2 = A[0,0] * B[0,0]:` This line calculates the product of the first and third elements of A and B.
  * `M3 = M1`: This line calculates the final result of the entire calculation by multiplying the first and third elements of A and B together.

Note that the values in the output matrix C are calculated using the same syntax as in the original input matrices. The resulting values are then assigned to the corresponding elements of the output matrix.

Result: INCORRECT
Length Mismatch: Expected 1 lines, Got 1 lines.
------------------------------
--- Test Case 10/10 ---
User Prompt

## More Epoch More Data - 25 epoch and 200 data

In [ ]:
import random
import json

# This COMPREHENSIVE_SYSTEM_MESSAGE_FOR_FULL_SEQUENCE defines all potential DSL rules.
# The fine-tuning data (assistant responses) will teach the specific algorithm and M/S naming conventions.
COMPREHENSIVE_SYSTEM_MESSAGE_FOR_FULL_SEQUENCE = """You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fine-tuning examples (assistant responses) will demonstrate the specific M-variable and S-variable naming convention (M for products, S for sums).
5.  If you cannot determine a valid full DSL sequence based on the rules and input, output ONLY the exact text: Error: Cannot determine full sequence.

DSL FORMAT DEFINITIONS:

1.  **Intermediate Variable Assignment (M-variables):**
    * Purpose: To store intermediate results from arithmetic operations or direct assignments.
    * Naming: `<id>` is an integer (e.g., M0, M1, M2, ...). Use the next available or contextually appropriate ID. (The specific M/S convention for matmul will be learned from examples).
    * Formats:
        * `M<id> = <operand1> + <operand2>`  (Addition)
        * `M<id> = <operand1> - <operand2>`  (Subtraction)
        * `M<id> = <operand1> * <operand2>`  (Multiplication of operands)
        * `M<id> = <operand>`             (Direct assignment)
    * Examples:
        * `M0 = A[0,0] + B[0,0]`
        * `M1 = A[0,1] - M0`
        * `M2 = A[0,0] * B[0,0]`
        * `M3 = M1`
        * `M4 = B[1,0]`

2.  **Output Matrix Cell Assignment (C-matrix):**
    * Purpose: To assign a computed value to an element of the 2x2 output matrix C.
    * Format: `C[<row>,<col>] = <operand>`
    * Constraints: `<row>` and `<col>` must be 0 or 1.
    * Example: `C[0,1] = M5` (Note: for standard matmul, the operand will typically be an S-variable as learned from examples like S0, S1, etc.)

3.  **Operands (`<operand>`, `<operand1>`, `<operand2>`):**
    * Can be an element from input matrix A: `A[<row>,<col>]` (e.g., A[0,0], A[1,0])
    * Can be an element from input matrix B: `B[<row>,<col>]` (e.g., B[0,1], B[1,1])
    * Can be a previously defined intermediate variable: `M<id>` (e.g., M0, M3). (The examples will show S-variables like S0, S1, etc., being used for sums in the matmul context).
    * Constraints: All `<row>` and `<col>` indices for A and B must be 0 or 1. Any `M<id>` (or S-variable, which is a type of intermediate variable) used as an operand must have been defined in a previous DSL step in the sequence.

You will be given the input matrices A (2x2) and B (2x2).
Provide the complete, multi-line DSL sequence to calculate C = A * B.
"""

def get_standard_ops_definition():
    """
    Defines the 16 DSL steps for a standard 2x2 matrix multiplication.
    The 'dsl' field provides the string for each step.
    This sequence implicitly defines the M-for-product and S-for-sum convention.
    """
    ops = [
        # C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0]
        {'dsl': "M0 = A[0,0]*B[0,0]", 'var_out': 'M0', 'symbolic_rhs': "A[0,0]*B[0,0]"},
        {'dsl': "M1 = A[0,1]*B[1,0]", 'var_out': 'M1', 'symbolic_rhs': "A[0,1]*B[1,0]"},
        {'dsl': "S0 = M0+M1",         'var_out': 'S0', 'symbolic_rhs': "M0+M1"},
        {'dsl': "C[0,0] = S0",        'var_out': None, 'c_idx': (0,0), 'symbolic_rhs': "S0"},

        # C[0,1] = A[0,0]*B[0,1] + A[0,1]*B[1,1]
        {'dsl': "M2 = A[0,0]*B[0,1]", 'var_out': 'M2', 'symbolic_rhs': "A[0,0]*B[0,1]"},
        {'dsl': "M3 = A[0,1]*B[1,1]", 'var_out': 'M3', 'symbolic_rhs': "A[0,1]*B[1,1]"},
        {'dsl': "S1 = M2+M3",         'var_out': 'S1', 'symbolic_rhs': "M2+M3"},
        {'dsl': "C[0,1] = S1",        'var_out': None, 'c_idx': (0,1), 'symbolic_rhs': "S1"},

        # C[1,0] = A[1,0]*B[0,0] + A[1,1]*B[1,0]
        {'dsl': "M4 = A[1,0]*B[0,0]", 'var_out': 'M4', 'symbolic_rhs': "A[1,0]*B[0,0]"},
        {'dsl': "M5 = A[1,1]*B[1,0]", 'var_out': 'M5', 'symbolic_rhs': "A[1,1]*B[1,0]"},
        {'dsl': "S2 = M4+M5",         'var_out': 'S2', 'symbolic_rhs': "M4+M5"},
        {'dsl': "C[1,0] = S2",        'var_out': None, 'c_idx': (1,0), 'symbolic_rhs': "S2"},

        # C[1,1] = A[1,0]*B[0,1] + A[1,1]*B[1,1]
        {'dsl': "M6 = A[1,0]*B[0,1]", 'var_out': 'M6', 'symbolic_rhs': "A[1,0]*B[0,1]"},
        {'dsl': "M7 = A[1,1]*B[1,1]", 'var_out': 'M7', 'symbolic_rhs': "A[1,1]*B[1,1]"},
        {'dsl': "S3 = M6+M7",         'var_out': 'S3', 'symbolic_rhs': "M6+M7"},
        {'dsl': "C[1,1] = S3",        'var_out': None, 'c_idx': (1,1), 'symbolic_rhs': "S3"},
    ]
    # Other keys like 'calc_val' are kept for potential other uses but not strictly needed
    # for this specific dataset generation task where only 'dsl' is used for the completion.
    return ops

def generate_random_matrix():
    """Generates a 2x2 matrix with random integer elements between -9 and 9."""
    return [[random.randint(-9, 9) for _ in range(2)] for _ in range(2)]

def generate_sft_dataset_full_sequence(num_full_multiplications=200, system_message=COMPREHENSIVE_SYSTEM_MESSAGE_FOR_FULL_SEQUENCE):
    """
    Generates a fine-tuning dataset for DSL-based 2x2 matrix multiplication.
    Each example is a full multiplication problem (A, B inputs) and the complete DSL sequence as completion.
    Formatted in ChatML.
    """
    ops_definition = get_standard_ops_definition()
    # The assistant's completion is the fixed sequence of DSL steps for standard 2x2 matmul
    full_dsl_completion = "\n".join([op['dsl'] for op in ops_definition])

    sft_data_chatml = [] # Store ChatML formatted strings

    for i in range(num_full_multiplications):
        # Generate new A and B matrices for each full multiplication problem
        if i == 0: current_A_val, current_B_val = [[1,0],[0,1]], generate_random_matrix()
        elif i == 1: current_A_val, current_B_val = generate_random_matrix(), [[1,0],[0,1]]
        elif i == 2: current_A_val, current_B_val = [[0,0],[0,0]], generate_random_matrix()
        elif i == 3: current_A_val, current_B_val = generate_random_matrix(), [[0,0],[0,0]]
        elif i == 4: current_A_val, current_B_val = [[-2, 3],[1, -4]], [[5, -1],[-2, 6]]
        else: current_A_val, current_B_val = generate_random_matrix(), generate_random_matrix()

        matrix_a_str_for_prompt = str(current_A_val).replace(" ", "")
        matrix_b_str_for_prompt = str(current_B_val).replace(" ", "")

        # Construct the "user" part of the prompt
        user_prompt_content = (
            f"Input matrices:\n"
            f"A = {matrix_a_str_for_prompt}\n"
            f"B = {matrix_b_str_for_prompt}\n"
            f"Provide the COMPLETE sequence of DSL steps to calculate C = A * B."
        )

        # Format into ChatML and add to dataset
        chatml_text = f"<|im_start|>system\n{system_message.strip()}<|im_end|>\n"
        chatml_text += f"<|im_start|>user\n{user_prompt_content.strip()}<|im_end|>\n"
        chatml_text += f"<|im_start|>assistant\n{full_dsl_completion}<|im_end|>"
        sft_data_chatml.append({"text": chatml_text})

    return sft_data_chatml

if __name__ == '__main__':
    num_multiplication_examples = 200 # Number of full 2x2 matrix multiplication problems for the dataset

    sft_dataset_chatml = generate_sft_dataset_full_sequence(
        num_multiplication_examples,
        system_message=COMPREHENSIVE_SYSTEM_MESSAGE_FOR_FULL_SEQUENCE
    )

    print(f"Generated {len(sft_dataset_chatml)} ChatML examples for SFT.")
    print(f"Each example provides input matrices A and B, and expects the full {len(get_standard_ops_definition())}-step DSL sequence as the completion.")

    print("\n----- Example SFT Data (First ChatML item) -----")
    if sft_dataset_chatml:
        print(sft_dataset_chatml[0]['text'])
        print("--------------------------------------------------")

    if len(sft_dataset_chatml) > 1:
        print("\n----- Example SFT Data (Second ChatML item, if generated) -----")
        print(sft_dataset_chatml[1]['text'])
        print("--------------------------------------------------")

    output_jsonl_path = "dsl_finetune_data_standard_matmul_full_sequence.jsonl"
    with open(output_jsonl_path, 'w') as f:
        for item in sft_dataset_chatml:
            f.write(json.dumps(item) + "\n")
    print(f"\nFine-tuning data saved to {output_jsonl_path}")

Generated 200 ChatML examples for SFT.
Each example provides input matrices A and B, and expects the full 16-step DSL sequence as the completion.

----- Example SFT Data (First ChatML item) -----
<|im_start|>system
You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fin

In [ ]:
# ==============================================
# QWEN DSL Fine-Tuning Full Script (Corrected)
# ==============================================
# Dependencies (install in your Colab / environment):
# !pip install -q datasets accelerate bitsandbytes trl

import os
import json
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig # TrainingArguments not strictly needed if all args in SFTConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training # PeftModel not used in training script
from trl import SFTConfig, SFTTrainer

os.environ["WANDB_DISABLED"] = "true"

# ------------------------------------------------
# 1. (Optional) Mount Google Drive
# ------------------------------------------------
# from google.colab import drive
# drive.mount('/content/drive')
# DRIVE_BASE_PATH = "/content/drive/MyDrive/Qwen_DSL_FineTune"
# os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

# ------------------------------------------------
# 2. System Message and Dataset Path
# ------------------------------------------------
SYSTEM_MESSAGE = (
   """You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fine-tuning examples (assistant responses) will demonstrate the specific M-variable and S-variable naming convention (M for products, S for sums).
5.  If you cannot determine a valid full DSL sequence based on the rules and input, output ONLY the exact text: Error: Cannot determine full sequence.

DSL FORMAT DEFINITIONS:

1.  **Intermediate Variable Assignment (M-variables):**
    * Purpose: To store intermediate results from arithmetic operations or direct assignments.
    * Naming: `<id>` is an integer (e.g., M0, M1, M2, ...). Use the next available or contextually appropriate ID. (The specific M/S convention for matmul will be learned from examples).
    * Formats:
        * `M<id> = <operand1> + <operand2>`  (Addition)
        * `M<id> = <operand1> - <operand2>`  (Subtraction)
        * `M<id> = <operand1> * <operand2>`  (Multiplication of operands)
        * `M<id> = <operand>`             (Direct assignment)
    * Examples:
        * `M0 = A[0,0] + B[0,0]`
        * `M1 = A[0,1] - M0`
        * `M2 = A[0,0] * B[0,0]`
        * `M3 = M1`
        * `M4 = B[1,0]`

2.  **Output Matrix Cell Assignment (C-matrix):**
    * Purpose: To assign a computed value to an element of the 2x2 output matrix C.
    * Format: `C[<row>,<col>] = <operand>`
    * Constraints: `<row>` and `<col>` must be 0 or 1.
    * Example: `C[0,1] = M5` (Note: for standard matmul, the operand will typically be an S-variable as learned from examples like S0, S1, etc.)

3.  **Operands (`<operand>`, `<operand1>`, `<operand2>`):**
    * Can be an element from input matrix A: `A[<row>,<col>]` (e.g., A[0,0], A[1,0])
    * Can be an element from input matrix B: `B[<row>,<col>]` (e.g., B[0,1], B[1,1])
    * Can be a previously defined intermediate variable: `M<id>` (e.g., M0, M3). (The examples will show S-variables like S0, S1, etc., being used for sums in the matmul context).
    * Constraints: All `<row>` and `<col>` indices for A and B must be 0 or 1. Any `M<id>` (or S-variable, which is a type of intermediate variable) used as an operand must have been defined in a previous DSL step in the sequence.

You will be given the input matrices A (2x2) and B (2x2).
Provide the complete, multi-line DSL sequence to calculate C = A * B."""
)

# Dataset path (ensure this file exists and is correctly formatted)
dataset_path = "/content/dsl_finetune_data_standard_matmul_full_sequence.jsonl"
if not os.path.exists(dataset_path):
  print(f"Dataset file not found: {dataset_path}. Please ensure it exists.")
  # Consider exiting or handling this error more robustly if the file is essential
  # exit()

# ------------------------------------------------
# 3. Load Dataset
# ------------------------------------------------
dataset = load_dataset("json", data_files=dataset_path, split="train")
print(f"Loaded {len(dataset)} examples from {dataset_path}")

# ------------------------------------------------
# 4. Model & Tokenizer Setup
# ------------------------------------------------
model_name = "Qwen/Qwen1.5-0.5B-Chat"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

# ------------------------------------------------
# 5. LoRA Configuration & PEFT Wrapping
# ------------------------------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ------------------------------------------------
# 6. Sanity Check: Ensure target_modules exist
# ------------------------------------------------
print("\nValidating target module names:")
targets = lora_config.target_modules
leafs = {name.split('.')[-1] for name, _ in model.named_modules()}
for t in targets:
    print(f"{t:10s} → {('FOUND' if t in leafs else 'MISSING')}")

# ------------------------------------------------
# 7. Training Configuration and Trainer Initialization
# ------------------------------------------------
output_dir = "./qwen_dsl_finetuned_adapter" # Base directory for all outputs

# SFTConfig inherits from TrainingArguments, so all parameters are set here.
sft_args = SFTConfig(
    # SFT specific parameters
    dataset_text_field="text",         # Field in your dataset that contains the ChatML string
    max_seq_length=1500,                # Max sequence length (corrected from max_length)
    packing=False,                     # Whether to pack multiple short examples if True

    # Standard TrainingArguments parameters integrated here
    output_dir=output_dir,             # Trainer will save checkpoints, logs etc. here
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,     # Effective batch size = 2 * 2 = 4
    learning_rate=1e-4,
    num_train_epochs=25,                # Increased epochs for small dataset (e.g., 25 examples). Adjust as needed.
    logging_steps=10,                  # Log training progress every 10 steps
    save_strategy="epoch",             # Save a checkpoint at the end of each epoch
    bf16=True,                         # Use bfloat16 precision (ensure GPU supports it)
    # fp16=False,                      # If bf16 not supported, set fp16=True
    optim="paged_adamw_8bit",          # Optimizer
    # report_to="none",                # Already handled by os.environ["WANDB_DISABLED"]
    # Add other relevant TrainingArguments as needed:
    # weight_decay=0.01,
    # lr_scheduler_type="cosine",
    # warmup_steps=50,
)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,                # Pass the tokenizer instance directly
    args=sft_args,                      # Pass the comprehensive SFTConfig object
    train_dataset=dataset,
    peft_config=lora_config,
    # dataset_text_field, max_seq_length, packing are sourced from sft_args now
)

# ------------------------------------------------
# 8. Start Fine-Tuning
# ------------------------------------------------
print("Starting fine-tuning with effective arguments...")
trainer.train() # Single, effective train call
print("Fine-tuning completed.")

# ------------------------------------------------
# 9. Save LoRA Adapter & Tokenizer
# ------------------------------------------------
# The adapter will be saved in a subdirectory of output_dir (e.g. output_dir/checkpoint-xxx or output_dir if save_strategy="no")
# To save the final adapter explicitly to a desired path after training:
adapter_path = os.path.join(output_dir, "final_adapter")
os.makedirs(adapter_path, exist_ok=True) # Ensure the directory exists
trainer.save_model(adapter_path)
tokenizer.save_pretrained(adapter_path)
print(f"Final adapter and tokenizer saved at {adapter_path}")

# ------------------------------------------------
# 10. (Optional) Test the Fine-Tuned Model (after training and saving)
# ------------------------------------------------
def test_model(prompt: str, trained_model, trained_tokenizer): # Pass the trained model and tokenizer
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user",   "content": prompt},
    ]
    text_in = trained_tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = trained_tokenizer([text_in], return_tensors="pt").to(trained_model.device)
    with torch.no_grad():
        out = trained_model.generate(
            **inputs,
            max_new_tokens=1500, # Increased from 25 to accommodate full DSL sequence
            do_sample=False,
            pad_token_id=trained_tokenizer.eos_token_id,
            eos_token_id=trained_tokenizer.eos_token_id
        )
    gen_tokens = out[0][inputs["input_ids"].shape[-1]:]
    return trained_tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

# To use the test_model function after training, you would typically load the saved adapter:
# from peft import PeftModel
#
# print("\n--- Testing the fine-tuned model ---")
# # Load the base model again (quantized)
# base_model_for_test = AutoModelForCausalLM.from_pretrained(
# model_name,
# quantization_config=bnb_config,
# device_map="auto",
# trust_remote_code=True
# )
# # Load the PeftModel with the adapter
# loaded_model = PeftModel.from_pretrained(base_model_for_test, adapter_path)
# loaded_model.eval() # Set to evaluation mode
#
# loaded_tokenizer = AutoTokenizer.from_pretrained(adapter_path) # Load tokenizer from adapter path
#
# test_prompt_example = (
#     "Input matrices:\n"
#     "A = [[1,2],[3,4]]\n"
#     "B = [[5,6],[7,8]]\n"
#     "Provide the COMPLETE sequence of DSL steps to calculate C = A * B."
# )
# response = test_model(test_prompt_example, loaded_model, loaded_tokenizer)
# print(f"Test Prompt:\n{test_prompt_example}")
# print(f"\nModel Response:\n{response}")

Generating train split: 0 examples [00:00, ? examples/s]

Loaded 200 examples from /content/dsl_finetune_data_standard_matmul_full_sequence.jsonl


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


trainable params: 7,569,408 || all params: 471,557,120 || trainable%: 1.6052

Validating target module names:
v_proj     → FOUND
up_proj    → FOUND
q_proj     → FOUND
gate_proj  → FOUND
o_proj     → FOUND
down_proj  → FOUND
k_proj     → FOUND


Converting train dataset to ChatML:   0%|          | 0/200 [00:00<?, ? examples/s]

...

Starting fine-tuning with effective arguments...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.731300
20,0.928600
30,0.282900
40,0.066700
50,0.029800
60,0.024800
70,0.023800
80,0.023000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


KeyboardInterrupt: 

#Adding Validation

In [ ]:
# ==============================================
# QWEN DSL Fine-Tuning Full Script (Corrected for SFTConfig & with Validation)
# ==============================================
# Dependencies (install in your Colab / environment):
# !pip install -q datasets accelerate bitsandbytes trl transformers # Ensure transformers is also listed

import os
import json
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig, TrainingArguments # Import TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer # SFTConfig might not be needed if passing args directly

os.environ["WANDB_DISABLED"] = "true"

# ------------------------------------------------
# 1. (Optional) Mount Google Drive
# ------------------------------------------------
# from google.colab import drive
# drive.mount('/content/drive')
# DRIVE_BASE_PATH = "/content/drive/MyDrive/Qwen_DSL_FineTune"
# os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

# ------------------------------------------------
# 2. System Message and Dataset Path
# ------------------------------------------------
SYSTEM_MESSAGE = (
   """You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fine-tuning examples (assistant responses) will demonstrate the specific M-variable and S-variable naming convention (M for products, S for sums).
5.  If you cannot determine a valid full DSL sequence based on the rules and input, output ONLY the exact text: Error: Cannot determine full sequence.

DSL FORMAT DEFINITIONS:

1.  **Intermediate Variable Assignment (M-variables):**
    * Purpose: To store intermediate results from arithmetic operations or direct assignments.
    * Naming: `<id>` is an integer (e.g., M0, M1, M2, ...). Use the next available or contextually appropriate ID. (The specific M/S convention for matmul will be learned from examples).
    * Formats:
        * `M<id> = <operand1> + <operand2>`  (Addition)
        * `M<id> = <operand1> - <operand2>`  (Subtraction)
        * `M<id> = <operand1> * <operand2>`  (Multiplication of operands)
        * `M<id> = <operand>`             (Direct assignment)
    * Examples:
        * `M0 = A[0,0] + B[0,0]`
        * `M1 = A[0,1] - M0`
        * `M2 = A[0,0] * B[0,0]`
        * `M3 = M1`
        * `M4 = B[1,0]`

2.  **Output Matrix Cell Assignment (C-matrix):**
    * Purpose: To assign a computed value to an element of the 2x2 output matrix C.
    * Format: `C[<row>,<col>] = <operand>`
    * Constraints: `<row>` and `<col>` must be 0 or 1.
    * Example: `C[0,1] = M5` (Note: for standard matmul, the operand will typically be an S-variable as learned from examples like S0, S1, etc.)

3.  **Operands (`<operand>`, `<operand1>`, `<operand2>`):**
    * Can be an element from input matrix A: `A[<row>,<col>]` (e.g., A[0,0], A[1,0])
    * Can be an element from input matrix B: `B[<row>,<col>]` (e.g., B[0,1], B[1,1])
    * Can be a previously defined intermediate variable: `M<id>` (e.g., M0, M3). (The examples will show S-variables like S0, S1, etc., being used for sums in the matmul context).
    * Constraints: All `<row>` and `<col>` indices for A and B must be 0 or 1. Any `M<id>` (or S-variable, which is a type of intermediate variable) used as an operand must have been defined in a previous DSL step in the sequence.

You will be given the input matrices A (2x2) and B (2x2).
Provide the complete, multi-line DSL sequence to calculate C = A * B."""
)

dataset_path = "/content/dsl_finetune_data_standard_matmul_full_sequence.jsonl"
if not os.path.exists(dataset_path):
  print(f"Dataset file not found: {dataset_path}. Please ensure it exists.")
  # exit()

# ------------------------------------------------
# 3. Load Dataset and Split for Validation
# ------------------------------------------------
full_dataset = load_dataset("json", data_files=dataset_path, split="train")
print(f"Loaded {len(full_dataset)} total examples from {dataset_path}")

full_dataset_shuffled = full_dataset.shuffle(seed=42)

if len(full_dataset_shuffled) >= 10:
    split_dataset = full_dataset_shuffled.train_test_split(test_size=0.2, seed=42)
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    print(f"Using {len(train_dataset)} examples for training and {len(eval_dataset)} for evaluation.")
else:
    print("Dataset too small for a meaningful train/test split. Using full dataset for both training and evaluation.")
    train_dataset = full_dataset_shuffled
    eval_dataset = full_dataset_shuffled

# ------------------------------------------------
# 4. Model & Tokenizer Setup
# ------------------------------------------------
model_name = "Qwen/Qwen1.5-0.5B-Chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model = prepare_model_for_kbit_training(model)

# ------------------------------------------------
# 5. LoRA Configuration & PEFT Wrapping
# ------------------------------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ------------------------------------------------
# 6. Sanity Check: Ensure target_modules exist
# ------------------------------------------------
print("\nValidating target module names:")
targets = lora_config.target_modules
leafs = {name.split('.')[-1] for name, _ in model.named_modules()}
for t in targets:
    print(f"{t:10s} → {('FOUND' if t in leafs else 'MISSING')}")

# ------------------------------------------------
# 7. Training Configuration and Trainer Initialization
# ------------------------------------------------
output_dir = "./qwen_dsl_finetuned_adapter"

from trl import SFTConfig, SFTTrainer

# 1. Build your SFTConfig (this replaces TrainingArguments for SFT-specific options)
sft_config = SFTConfig(
    # Tell it which column in your dataset holds the raw text
    dataset_text_field="text",
    # How long to truncate / pad sequences
    max_seq_length=2048,
    # Whether to pack multiple examples into one batch element
    packing=True,

    # You can also include Trainer hyperparams here:
    output_dir="./qwen_dsl_finetuned_adapter",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_8bit",
    report_to=["none"],
)

# 2. Instantiate SFTTrainer using that config
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # if you have an eval split
    processing_class=tokenizer,
    peft_config=lora_config,
)

# 3. Train!
trainer.train()

# ------------------------------------------------
# 8. Start Fine-Tuning
# ------------------------------------------------
print("Starting fine-tuning with effective arguments and evaluation...")
trainer.train()
print("Fine-tuning completed.")

# ------------------------------------------------
# 9. Save LoRA Adapter & Tokenizer
# ------------------------------------------------
adapter_path = os.path.join(output_dir, "final_adapter_after_full_train")
os.makedirs(adapter_path, exist_ok=True)
trainer.save_model(adapter_path)
tokenizer.save_pretrained(adapter_path)
print(f"Final adapter and tokenizer (after full training) saved at {adapter_path}")

# ------------------------------------------------
# 10. (Optional) Test the Fine-Tuned Model
# ------------------------------------------------
def test_model(prompt: str, trained_model, trained_tokenizer):
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user",   "content": prompt},
    ]
    text_in = trained_tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = trained_tokenizer([text_in], return_tensors="pt").to(trained_model.device)
    with torch.no_grad():
        out = trained_model.generate(
            **inputs,
            max_new_tokens=1500,
            do_sample=False,
            pad_token_id=trained_tokenizer.eos_token_id,
            eos_token_id=trained_tokenizer.eos_token_id
        )
    gen_tokens = out[0][inputs["input_ids"].shape[-1]:]
    return trained_tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

# Example of how to test after training
# (Ensure bnb_config, model_name, SYSTEM_MESSAGE are available if running this part)
# from peft import PeftModel
# print("\n--- Potentially testing the fine-tuned model ---")
# adapter_to_test_path = adapter_path
#
# try:
#     base_model_for_test = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         quantization_config=bnb_config,
#         device_map="auto",
#         trust_remote_code=True
#     )
#     print(f"Loading adapter from: {adapter_to_test_path}")
#     loaded_model = PeftModel.from_pretrained(base_model_for_test, adapter_to_test_path)
#     loaded_model.eval()
#     loaded_tokenizer = AutoTokenizer.from_pretrained(adapter_to_test_path)
#
#     test_prompt_example = (
#         "Input matrices:\n"
#         "A = [[1,2],[3,4]]\n"
#         "B = [[5,6],[7,8]]\n"
#         "Provide the COMPLETE sequence of DSL steps to calculate C = A * B."
#     )
#     response = test_model(test_prompt_example, loaded_model, loaded_tokenizer)
#     print(f"Test Prompt:\n{test_prompt_example}")
#     print(f"\nModel Response:\n{response}")
# except Exception as e:
#     print(f"Could not run optional test: {e}")

Loaded 200 total examples from /content/dsl_finetune_data_standard_matmul_full_sequence.jsonl
Using 160 examples for training and 40 for evaluation.
trainable params: 7,569,408 || all params: 471,557,120 || trainable%: 1.6052

Validating target module names:
v_proj     → FOUND
up_proj    → FOUND
q_proj     → FOUND
gate_proj  → FOUND
o_proj     → FOUND
down_proj  → FOUND
k_proj     → FOUND


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Starting fine-tuning with effective arguments and evaluation...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Fine-tuning completed.
Final adapter and tokenizer (after full training) saved at ./qwen_dsl_finetuned_adapter/final_adapter_after_full_train


In [ ]:
 import os
import json
import torch
import random
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig
)
from peft import PeftModel

# Ensure WANDB is disabled if you had it enabled previously for training
os.environ["WANDB_DISABLED"] = "true"

# ------------------------------------------------
# Configuration (Adjust these paths if necessary)
# ------------------------------------------------
BASE_MODEL_NAME = "Qwen/Qwen1.5-0.5B-Chat"
# This is the directory where trainer.save_model() saved the adapter
ADAPTER_PATH = "./qwen_dsl_finetuned_adapter/final_adapter" # Output from step 9

# ------------------------------------------------
# System Message (MUST be IDENTICAL to the one used for fine-tuning)
# ------------------------------------------------
SYSTEM_MESSAGE = (
   """You are an AI assistant. Your ONLY task is to output the COMPLETE sequence of VALID DSL lines to perform a 2x2 matrix multiplication (C = A * B), with each DSL line on a new line.
STRICT RULES:
1.  Adhere strictly to the DSL formats defined below for each line in the sequence.
2.  NO PYTHON code (e.g., 'np.dot', '@', 'np.array', 'loops').
3.  NO EXPLANATIONS, apologies, or conversational text. Output only the DSL lines, each on a new line.
4.  The generated sequence must correctly implement the standard 2x2 matrix multiplication algorithm as a sequence of elementary operations.
    (e.g., C[0,0] = A[0,0]*B[0,0] + A[0,1]*B[1,0] should be broken down into steps like M0=A[0,0]*B[0,0], M1=A[0,1]*B[1,0], S0=M0+M1, C[0,0]=S0, following this pattern for all C elements).
    The fine-tuning examples (assistant responses) will demonstrate the specific M-variable and S-variable naming convention (M for products, S for sums).
5.  If you cannot determine a valid full DSL sequence based on the rules and input, output ONLY the exact text: Error: Cannot determine full sequence.

DSL FORMAT DEFINITIONS:

1.  **Intermediate Variable Assignment (M-variables):**
    * Purpose: To store intermediate results from arithmetic operations or direct assignments.
    * Naming: `<id>` is an integer (e.g., M0, M1, M2, ...). Use the next available or contextually appropriate ID. (The specific M/S convention for matmul will be learned from examples).
    * Formats:
        * `M<id> = <operand1> + <operand2>`  (Addition)
        * `M<id> = <operand1> - <operand2>`  (Subtraction)
        * `M<id> = <operand1> * <operand2>`  (Multiplication of operands)
        * `M<id> = <operand>`             (Direct assignment)
    * Examples:
        * `M0 = A[0,0] + B[0,0]`
        * `M1 = A[0,1] - M0`
        * `M2 = A[0,0] * B[0,0]`
        * `M3 = M1`
        * `M4 = B[1,0]`

2.  **Output Matrix Cell Assignment (C-matrix):**
    * Purpose: To assign a computed value to an element of the 2x2 output matrix C.
    * Format: `C[<row>,<col>] = <operand>`
    * Constraints: `<row>` and `<col>` must be 0 or 1.
    * Example: `C[0,1] = M5` (Note: for standard matmul, the operand will typically be an S-variable as learned from examples like S0, S1, etc.)

3.  **Operands (`<operand>`, `<operand1>`, `<operand2>`):**
    * Can be an element from input matrix A: `A[<row>,<col>]` (e.g., A[0,0], A[1,0])
    * Can be an element from input matrix B: `B[<row>,<col>]` (e.g., B[0,1], B[1,1])
    * Can be a previously defined intermediate variable: `M<id>` (e.g., M0, M3). (The examples will show S-variables like S0, S1, etc., being used for sums in the matmul context).
    * Constraints: All `<row>` and `<col>` indices for A and B must be 0 or 1. Any `M<id>` (or S-variable, which is a type of intermediate variable) used as an operand must have been defined in a previous DSL step in the sequence.

You will be given the input matrices A (2x2) and B (2x2).
Provide the complete, multi-line DSL sequence to calculate C = A * B."""
)

# ------------------------------------------------
# Helper functions from your data generation script
# ------------------------------------------------
def get_standard_ops_definition():
    """
    Defines the 16 DSL steps for a standard 2x2 matrix multiplication.
    """
    ops = [
        {'dsl': "M0 = A[0,0]*B[0,0]"}, {'dsl': "M1 = A[0,1]*B[1,0]"},
        {'dsl': "S0 = M0+M1"},         {'dsl': "C[0,0] = S0"},
        {'dsl': "M2 = A[0,0]*B[0,1]"}, {'dsl': "M3 = A[0,1]*B[1,1]"},
        {'dsl': "S1 = M2+M3"},         {'dsl': "C[0,1] = S1"},
        {'dsl': "M4 = A[1,0]*B[0,0]"}, {'dsl': "M5 = A[1,1]*B[1,0]"},
        {'dsl': "S2 = M4+M5"},         {'dsl': "C[1,0] = S2"},
        {'dsl': "M6 = A[1,0]*B[0,1]"}, {'dsl': "M7 = A[1,1]*B[1,1]"},
        {'dsl': "S3 = M6+M7"},         {'dsl': "C[1,1] = S3"},
    ]
    return ops

def generate_random_matrix():
    """Generates a 2x2 matrix with random integer elements between -9 and 9."""
    return [[random.randint(-9, 9) for _ in range(2)] for _ in range(2)]

EXPECTED_DSL_SEQUENCE = "\n".join([op['dsl'] for op in get_standard_ops_definition()])

# ------------------------------------------------
# Load Fine-Tuned Model for Inference
# ------------------------------------------------
def load_model_for_inference(base_model_name, adapter_path):
    print(f"Loading base model: {base_model_name}")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, # Or 8bit if you trained with that
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config, # Use same quantization as training for base
        device_map="auto",
        trust_remote_code=True
    )

    print(f"Loading adapter from: {adapter_path}")
    # Load the PEFT model by specifying the adapter path
    # The `is_trainable=False` is good practice for inference.
    model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)

    print("Merging adapter into the base model...")
    # Merge LoRA layers for faster inference.
    # If you run out of memory, you can skip this and use the PeftModel directly,
    # but inference will be slower.
    model = model.merge_and_unload()

    tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token # Or tokenizer.unk_token
    tokenizer.padding_side = "right" # Important for generation

    print("Model and tokenizer loaded successfully.")
    return model, tokenizer

# ------------------------------------------------
# Generate Model Response
# ------------------------------------------------
def get_model_response(model, tokenizer, user_prompt_content):
    messages = [
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": user_prompt_content},
    ]

    # Apply chat template
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True # Crucial for Qwen and other instruction-tuned models
    )

    inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

    # Generation parameters
    # max_new_tokens needs to be sufficient for the 16 lines of DSL.
    # 16 lines * ~20 chars/line = 320. Add some buffer.
    # Using do_sample=False for deterministic output, good for testing exact match.
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2000, # Increased for full sequence
            do_sample=False,    # For deterministic output
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id # Model should stop after generating its response
        )

    # Decode only the generated part
    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    response_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response_text.strip()

# ------------------------------------------------
# Main Testing Logic
# ------------------------------------------------
if __name__ == "__main__":
    if not os.path.exists(ADAPTER_PATH):
        print(f"ERROR: Adapter path not found: {ADAPTER_PATH}")
        print("Please ensure you have trained the model and the adapter is saved correctly.")
        exit(1)

    model, tokenizer = load_model_for_inference(BASE_MODEL_NAME, ADAPTER_PATH)
    model.eval() # Set model to evaluation mode

    num_test_cases = 10
    correct_predictions = 0

    print(f"\n--- Starting {num_test_cases} Test Cases ---")
    print(f"Expected DSL Sequence (all test cases):\n{EXPECTED_DSL_SEQUENCE}\n")

    for i in range(num_test_cases):
        print(f"--- Test Case {i+1}/{num_test_cases} ---")

        # Generate random matrices for the prompt
        matrix_a = generate_random_matrix()
        matrix_b = generate_random_matrix()

        matrix_a_str = str(matrix_a).replace(" ", "")
        matrix_b_str = str(matrix_b).replace(" ", "")

        user_prompt = (
            f"Input matrices:\n"
            f"A = {matrix_a_str}\n"
            f"B = {matrix_b_str}\n"
            f"Provide the COMPLETE sequence of DSL steps to calculate C = A * B."
        )
        print(f"User Prompt (content part):\n{user_prompt}")

        generated_dsl = get_model_response(model, tokenizer, user_prompt)

        print(f"\nModel's Generated DSL:\n{generated_dsl}")

        # Normalize whitespace for comparison (e.g., handle potential trailing newlines)
        normalized_generated = "\n".join([line.strip() for line in generated_dsl.strip().split('\n')])
        normalized_expected = "\n".join([line.strip() for line in EXPECTED_DSL_SEQUENCE.strip().split('\n')])

        if normalized_generated == normalized_expected:
            print("\nResult: CORRECT")
            correct_predictions += 1
        else:
            print("\nResult: INCORRECT")
            # For detailed diff, you could use difflib
            # import difflib
            # diff = difflib.unified_diff(
            #     normalized_expected.splitlines(keepends=True),
            #     normalized_generated.splitlines(keepends=True),
            #     fromfile='expected',
            #     tofile='generated',
            # )
            # print("Diff:")
            # print(''.join(diff))
            if len(normalized_generated.split('\n')) != len(normalized_expected.split('\n')):
                print(f"""Length Mismatch: Expected {len(normalized_expected.split('     '))} lines, Got {len(normalized_generated.split('     '))} lines.""")


        print("-" * 30)

    accuracy = (correct_predictions / num_test_cases) * 100
    print(f"\n--- Test Summary ---")
    print(f"Total Test Cases: {num_test_cases}")
    print(f"Correct Predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")

    # Clean up model from GPU memory if necessary
    del model
    del tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

Loading base model: Qwen/Qwen1.5-0.5B-Chat
Loading adapter from: ./qwen_dsl_finetuned_adapter/final_adapter
Merging adapter into the base model...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model and tokenizer loaded successfully.

--- Starting 10 Test Cases ---
Expected DSL Sequence (all test cases):
M0 = A[0,0]*B[0,0]
M1 = A[0,1]*B[1,0]
S0 = M0+M1
C[0,0] = S0
M2 = A[0,0]*B[0,1]
M3 = A[0,1]*B[1,1]
S1 = M2+M3
C[0,1] = S1
M4 = A[1,0]*B[0,0]
M5 = A[1,1]*B[1,0]
S2 = M4+M5
C[1,0] = S2
M6 = A[1,0]*B[0,1]
M7 = A[1,1]*B[1,1]
S3 = M6+M7
C[1,1] = S3

--- Test Case 1/10 ---
User Prompt (content part):
Input matrices:
A = [[-6,-9],[-1,-2]]
B = [[-2,-5],[-6,8]]
Provide the COMPLETE sequence of DSL steps to calculate C = A * B.


The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Model's Generated DSL:
Here's the complete sequence of DSL steps to calculate C = A * B:
```
M0 = A[0,0] + B[0,0]
M1 = A[0,1] - M0
M2 = A[0,0] * B[0,0]
M3 = M1
S0 = M0 + M1
C = M0 * M1
```

Explanation:

  * `M0 = A[0,0] + B[0,0]`: This line calculates the sum of the first two elements of the input matrices A and B.
  * `M1 = A[0,1] - M0`: This line calculates the difference between the second and third elements of the input matrices A and B.
  * `M2 = A[0,0] * B[0,0]:` This line calculates the product of the first and third elements of the input matrices A and B.
  * `M3 = M1`: This line calculates the final result of the calculation, which is the computed value of A * B.

Note that the values returned by these steps are not necessarily the same as the actual values calculated by the matmul operation. However, they represent the computed value of A * B.

Result: INCORRECT
Length Mismatch: Expected 1 lines, Got 1 lines.
------------------------------
--- Test Case 2/10 ---
User Prompt

...

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Model's Generated DSL:
Here's the complete sequence of DSL steps to calculate C = A * B:
```
M0 = A[0,0] + B[0,0]
M1 = A[0,1] - M0
M2 = A[0,0] * B[0,0]
M3 = M1
S0 = M0 + M1
C = M0 * M1
```

Explanation:

  * `M0 = A[0,0] + B[0,0]`: This line calculates the sum of the first two columns of A and the second column of B.
  * `M1 = A[0,1] - M0`: This line calculates the difference between the first two columns of A and the second column of B.
  * `M2 = A[0,0] * B[0,0]:` This line calculates the product of the first two columns of A and the second column of B.
  * `M3 = M1`: This line assigns the result of the previous step to the current element of the output matrix C.
  * `S0 = M0 + M1`: This line adds the values of M0 and M1 together to get the final value of C.
  * `C = M0 * M1`: This line combines the values of M0 and M1 to get the final value of C.

Note that the values of M0 and M1 are determined by the input matrices A and B, respectively. These values can be adjusted as needed to m

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Model's Generated DSL:
Here's the complete sequence of DSL steps to calculate C = A * B:
```
M0 = A[0,0] + B[0,0]
M1 = A[0,1] - M0
M2 = A[0,0] * B[0,0]
M3 = M1
S0 = M0 + M1
C = M0 * M1
```

Explanation:

  * `M0 = A[0,0] + B[0,0]`: This line calculates the sum of the first two columns of A and the second column of B.
  * `M1 = A[0,1] - M0`: This line calculates the difference between the first two columns of A and the second column of B.
  * `M2 = A[0,0] * B[0,0]:` This line calculates the product of the first two columns of A and the second column of B.
  * `M3 = M1`: This line assigns the result of the previous step to the current element of the output matrix C.
  * `S0 = M0 + M1`: This line adds the values of M0 and M1 together to get the final value of C.
  * `C = M0 * M1`: This line combines the values of M0 and M1 to get the final value of C.

Note that the values of M0 and M1 are determined by the input matrices A and B, respectively. These values can be adjusted as needed to m

KeyboardInterrupt: 

In [ ]:
import torch, gc

# 3. Force a full GC pass
gc.collect()

# 4. Return any freed blocks to CUDA
torch.cuda.empty_cache()

# 5. (Optional) Check current usage
print(torch.cuda.memory_summary())